In [1]:
# This notebook contains the basic plots in the GBO drone paper

In [2]:
## First import general packages for running python analysis:
import os, h5py, datetime,pytz, yaml
import numpy as np
from matplotlib.pyplot import *
from matplotlib import pyplot as plt
import pickle, glob

from scipy.interpolate import griddata
import scipy.optimize as opt
from scipy.optimize import curve_fit
from scipy.optimize import leastsq
from scipy.optimize import least_squares
from matplotlib import cm


from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.ticker import MultipleLocator

import csv
from math import *

## Then import the beamcals module packages and initialize 'gbosite' class:
from beamcals import corr
from beamcals import concat
from beamcals import drone
from beamcals import bicolog
import beamcals.plotting_utils as pu
import beamcals.fitting_utils as fu
import beamcals.geometry_utils as gu
import beamcals.time_utils as tu
from beamcals.sites import site
from beamcals import beammap as bp
gbosite=site.site('../beamcals/beamcals/sites/GBO_config.npz')

# various gridding attempts
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging


In [3]:
# Defining anything I want to keep the same 

freqs = 800.0*np.ones(1024) + (-400/1024.)*np.arange(1024)


#basedir = '/hirax/GBO_Analysis_Outputs/2023_SpringSummer_products/'
basedir = '/hirax/GBO_Analysis_Outputs/'
fitdir=basedir+'main_beam_fits/'
ymldir = basedir+'concat_config_files/'
pckldir = basedir+'flight_pickles/'
ampdir = basedir+'amplitude_corrections/'
beamdir = basedir+'beam_pickles/'
polbeamdir = '/hirax/GBO_Analysis_Outputs/beam_pickles_polar/'

dronedir='/hirax/all_drone_data/datcon_csv/'
fltyaml = '/hirax/GBO_Analysis_Outputs/GBO_flights_forscripts.yaml'


# Plotting standards
fsz = 16 # fontsize for labels and legends
ccm = cm.magma # use magma for color maps


In [4]:
## Useful functions
def get_flightinfo(fly):
    with open('/hirax/GBO_Analysis_Outputs/GBO_flights_forscripts.yaml', 'r') as fff:
        documents = yaml.safe_load(fff)
    flights = documents["flight_info"]["flights"]
    
    for fi in range(0,len(documents["flight_info"]["flights"])):
        if fly == flights[fi]: f = fi
    
    if polarn[f]=='N':
        pols = N_pols
        cpols = E_pols
    else:
        pols = E_pols
        cpols = N_pols
    return polarn[f], pols, cpols, attns[f], f


def convert_to_angle(d):
    h = 177.0 # meters
    aa = np.arctan(d/h)
    return np.degrees(aa)

def convert_sig_to_FWHM(sig):
    return 2.355*sig

def get_polar_slice(theta,phi,val=0.0): # assume val in angle
    
    tol = abs(theta[1,0] - theta[0,0])/1.5
    N = len(theta[:,0]) #figure out the importance of this 
    #ok = True
    #while(ok):
    sliceIndex1 = np.where(
            (theta[:,0] < (val + tol)) & 
            (theta[:,0] > (val - tol)))[0][0]
    sliceIndex2 = np.where(
            (theta[:,0] < (np.pi + val + tol)) & 
            (theta[:,0] > (np.pi + val - tol)))[0][0]
    return sliceIndex1, sliceIndex2


In [5]:
with open('/hirax/GBO_Analysis_Outputs/GBO_flights_forscripts.yaml', 'r') as fff:
    documents = yaml.safe_load(fff)
flights = documents["flight_info"]["flights"]
N_pols = documents["flag_info"]["N_pols"]
E_pols = documents["flag_info"]["E_pols"]
polarn = documents["flight_info"]["pols"]
attns = documents["flight_info"]["attns"]
masks = documents["flight_info"]["masks"]
good_freqs = documents["freq_info"]["good_freqs"]

pcklarr=np.sort(os.listdir(pckldir))
gfitarr=np.sort(os.listdir(fitdir))
amparr=np.sort(os.listdir(ampdir))
print(gfitarr,amparr)

Npolflights = ['618','619','625','646','647','533','536']
Epolflights = ['620','648','649','535']
NF = len(Npolflights)
NE = len(Epolflights)
F = NF+NE
fi_i =650

find=900



['FLY533_20210826T182904Z_2dGauss_and_Airy_Params.npz'
 'FLY534_20210826T185621Z_2dGauss_and_Airy_Params.npz'
 'FLY535_20210826T192056Z_2dGauss_and_Airy_Params.npz'
 'FLY536_20210826T195003Z_2dGauss_and_Airy_Params.npz'
 'FLY618_20211021T183430Z_2dGauss_and_Airy_Params.npz'
 'FLY619_20211021T185805Z_2dGauss_and_Airy_Params.npz'
 'FLY620_20211021T192212Z_2dGauss_and_Airy_Params.npz'
 'FLY623_20211021T203206Z_2dGauss_and_Airy_Params.npz'
 'FLY625_20211021T214028Z_2dGauss_and_Airy_Params.npz'
 'FLY646_20211022T183659Z_2dGauss_and_Airy_Params.npz'
 'FLY647_20211022T185749Z_2dGauss_and_Airy_Params.npz'
 'FLY648_20211022T191922Z_2dGauss_and_Airy_Params.npz'
 'FLY649_20211022T194308Z_2dGauss_and_Airy_Params.npz'] ['FLY533_Corrected_amplitudes.pkl' 'FLY534_Corrected_amplitudes.pkl'
 'FLY535_Corrected_amplitudes.pkl' 'FLY536_Corrected_amplitudes.pkl'
 'FLY618_Corrected_amplitudes.pkl' 'FLY619_Corrected_amplitudes.pkl'
 'FLY620_Corrected_amplitudes.pkl' 'FLY623_Corrected_amplitudes.pkl'
 'FLY625

In [6]:
print(flights)

['533', '534', '535', '536', '618', '619', '620', '623', '625', '646', '647', '648', '649']


####################################################################################
############################# SECTION 2 ##########################################
############################# SECTION 2 ##########################################
############################# SECTION 2 ##########################################
############################# SECTION 2 ##########################################
####################################################################################

In [ ]:
####################################################################################
############################# SECTION 2.1 ##########################################
############################# SECTION 2.1 ##########################################
####################################################################################


######################################################################
######################################################################
########## Comparing Gaussian centroids between flights ##########
######################################################################
######################################################################

print('Finding the median pointing per flight, and the average difference from the median')

############ Npol
pols = N_pols
flies = Npolflights
allfits = np.zeros([len(pols),1024,2,len(flies)]) # dish, freq, x/y, flight

#fig = plt.figure(figsize=(15,10))
for f,fly in enumerate(flies):
    fits = np.load(glob.glob(fitdir+'*'+fly+'*')[0])
    for p,pol in enumerate(pols):
        allfits[p,:,0,f] = fits['G_popt'][pol,:,1]
        allfits[p,:,1,f] = fits['G_popt'][pol,:,3]        

        
# First, generate the median value per flight, we need this to do some differencing
xmeds = np.nanmedian(allfits[:,fi_i:-1,0,:],axis=-1)
ymeds = np.nanmedian(allfits[:,fi_i:-1,1,:],axis=-1)

meddif_pppf = np.zeros([len(pols),NF,3])
meddif_pf = np.zeros([F,3])

for f,fly in enumerate(flies):
    for p,pol in enumerate(pols):
        meddif_pppf[p,f,0] = np.nanmean(allfits[p,fi_i:-1,0,f] - xmeds[p,:]) #x
        meddif_pppf[p,f,1] = np.nanmean(allfits[p,fi_i:-1,1,f] - ymeds[p,:]) #y
        meddif_pppf[p,f,2] = np.nanmean(np.sqrt((allfits[p,fi_i:-1,0,f] - xmeds[p,:])**2 + 
                                           (allfits[p,fi_i:-1,1,f] - ymeds[p,:])**2)) #r

meddif_pf[0:NF,:] = np.nanmean(meddif_pppf[:,:,:],axis=0)

print('per flight X   perflight Y   perflight R')
for f, fly in enumerate(flies):
    print(fly, meddif_pf[f,:])

    
############ Epol
pols = E_pols
flies = Epolflights
allfits = np.zeros([len(pols),1024,2,len(flies)]) # dish, freq, x/y, flight

#fig = plt.figure(figsize=(15,10))
for f,fly in enumerate(flies):
    fits = np.load(glob.glob(fitdir+'*'+fly+'*')[0])
    for p,pol in enumerate(pols):
        allfits[p,:,0,f] = fits['G_popt'][pol,:,1]
        allfits[p,:,1,f] = fits['G_popt'][pol,:,3]        

        
# First, generate the median value per flight, we need this to do some differencing
xmeds = np.nanmedian(allfits[:,fi_i:-1,0,:],axis=-1)
ymeds = np.nanmedian(allfits[:,fi_i:-1,1,:],axis=-1)

meddif_pppf = np.zeros([len(pols),len(flies),3])
for f,fly in enumerate(flies):
    for p,pol in enumerate(pols):
        meddif_pppf[p,f,0] = np.nanmedian(allfits[p,fi_i:-1,0,f] - xmeds[p,:])
        meddif_pppf[p,f,1] = np.nanmedian(allfits[p,fi_i:-1,1,f] - ymeds[p,:])
        meddif_pppf[p,f,2] = np.nanmedian(np.sqrt((allfits[p,fi_i:-1,0,f] - xmeds[p,:])**2 + 
                                           (allfits[p,fi_i:-1,1,f] - ymeds[p,:])**2))   

meddif_pf[NF:F,:] = np.nanmedian(meddif_pppf[:,:,:],axis=0)

for f, fly in enumerate(flies):
    print(fly, meddif_pf[NF+f,:])    

print('AVG X   AVG Y   AVG R')
print(np.nanmedian(meddif_pf[:,0]),np.nanmedian(meddif_pf[:,1]),np.nanmedian(meddif_pf[:,2]))
        
augs = [5,6,10]
print('AUG:: AVG X   AVG Y   AVG R')
print(np.nanmedian(meddif_pf[augs,0]),np.nanmedian(meddif_pf[augs,1]),np.nanmedian(meddif_pf[augs,2]))

octs = [0,1,2,3,4,7,8,9,10]
print('OCT:: AVG X   AVG Y   AVG R')
print(np.nanmedian(meddif_pf[octs,0]),np.nanmedian(meddif_pf[octs,1]),np.nanmedian(meddif_pf[octs,2]))

######################################################################
######################################################################
########## Comparing Gaussian centroids  to ref flight ##########
######################################################################
######################################################################

print('Finding the median pointing per flight compared to a reference flight (618, 620)')

###### Npols 
pols = N_pols
flies = Npolflights
allfits = np.zeros([len(pols),1024,2,len(flies)]) # dish, freq, x/y, flight

#fig = plt.figure(figsize=(15,10))
for f,fly in enumerate(flies):
    fits = np.load(glob.glob(fitdir+'*'+fly+'*')[0])
    for p,pol in enumerate(pols):
        allfits[p,:,0,f] = fits['G_popt'][pol,:,1]
        allfits[p,:,1,f] = fits['G_popt'][pol,:,3]        

refdif_pppf = np.zeros([len(pols),NF-1,3]) # average and std of differences [pol,flights, med/std]
refdif_pf = np.zeros([F,3])

# Find difference between flight 618 and the rest, per input
for f in np.arange(1,NF):
    for p,pol in enumerate(pols):
        refdif_pppf[p,f-1,0] = np.nanmedian(allfits[p,fi_i:-1,0,f] - allfits[p,fi_i:-1,0,0]) # x
        refdif_pppf[p,f-1,1] = np.nanmedian(allfits[p,fi_i:-1,1,f] - allfits[p,fi_i:-1,1,0]) # y
        refdif_pppf[p,f-1,2] = np.nanmedian(np.sqrt((allfits[p,fi_i:-1,0,f] - allfits[p,fi_i:-1,0,0])**2 +
                                       (allfits[p,fi_i:-1,1,f] - allfits[p,fi_i:-1,1,0])**2))

refdif_pf[1:NF,:] = np.nanmean(refdif_pppf[:,:,:],axis=0)

###### Epols
pols = E_pols
flies = Epolflights
allfits = np.zeros([len(pols),1024,2,len(flies)]) # dish, freq, x/y, flight

#fig = plt.figure(figsize=(15,10))
for f,fly in enumerate(flies):
    fits = np.load(glob.glob(fitdir+'*'+fly+'*')[0])
    for p,pol in enumerate(pols):
        allfits[p,:,0,f] = fits['G_popt'][pol,:,1]
        allfits[p,:,1,f] = fits['G_popt'][pol,:,3]        
        
refdif_pppf = np.zeros([len(pols),NE-1,3]) # average and std of differences [pol,flights, med/std]

# Find difference between flight 618 and the rest, per input
for f in np.arange(1,NE):
    for p,pol in enumerate(pols):
        refdif_pppf[p,f-1,0] = np.nanmedian(allfits[p,fi_i:-1,0,f] - allfits[p,fi_i:-1,0,0]) # x
        refdif_pppf[p,f-1,1] = np.nanmedian(allfits[p,fi_i:-1,1,f] - allfits[p,fi_i:-1,1,0]) # y
        refdif_pppf[p,f-1,2] = np.nanmedian(np.sqrt((allfits[p,fi_i:-1,0,f] - allfits[p,fi_i:-1,0,0])**2 +
                                       (allfits[p,fi_i:-1,1,f] - allfits[p,fi_i:-1,1,0])**2))

        

refdif_pf[NF+1:F,:] = np.nanmedian(refdif_pppf[:,:,:],axis=0)
    

nonzero = [1,2,3,4,5,6,8,9,10]    
print('AVG X   AVG Y   AVG R')
print(np.nanmedian(refdif_pf[nonzero,0]),np.nanmedian(refdif_pf[nonzero,1]),np.nanmedian(refdif_pf[nonzero,2]))
        
######################################################################
######################################################################
########## Comparing Return to points between flights ##########
######################################################################
######################################################################

print('Comparing the return to point *between* flights')


# This cell reads in all of the drone data for all of the flights, so it will take a few minutes 

#Npolflight order
tlbsa = [0,0,0,0,0,0,0]
tubsa = [1000,350,600,1000,100,400,350]
tlba = [6000,5400,6570,6480,5640,6200,5850]
tuba = [-1,-1,-1,-1,-1,-1,-1]

Nrtp = np.zeros([len(Npolflights),4,2]) # flight, startX/endY/startY/endY, med/std

for f,fly in enumerate(Npolflights):
    dronedir='/hirax/all_drone_data/datcon_csv/'
    dronedat=drone.Drone_Data(Drone_Directory=dronedir,FLYTAG='FLY'+fly+'.csv',site_class=gbosite)

    
    Nrtp[f,0,0] = np.nanmedian(dronedat.coords_xyz_LC[tlbsa[f]:tubsa[f],0]) # start X
    Nrtp[f,1,0] = np.nanmedian(dronedat.coords_xyz_LC[tlba[f]:tuba[f],0])  # end X
    Nrtp[f,2,0] = np.nanmedian(dronedat.coords_xyz_LC[tlbsa[f]:tubsa[f],1]) # start Y
    Nrtp[f,3,0] = np.nanmedian(dronedat.coords_xyz_LC[tlba[f]:tuba[f],1]) #end Y

    Nrtp[f,0,1] = np.nanstd(dronedat.coords_xyz_LC[tlbsa[f]:tubsa[f],0])
    Nrtp[f,1,1] = np.nanstd(dronedat.coords_xyz_LC[tlba[f]:tuba[f],0])
    Nrtp[f,2,1] = np.nanstd(dronedat.coords_xyz_LC[tlbsa[f]:tubsa[f],1])
    Nrtp[f,3,1] = np.nanstd(dronedat.coords_xyz_LC[tlba[f]:tuba[f],1])

    
print('Flight  delX    delY  delR ---- Npolflights')
for f in np.arange(len(Npolflights)):
    print(Npolflights[f],(Nrtp[f,0,0]-Nrtp[f,1,0]),Nrtp[f,2,0]-Nrtp[f,3,0],
          np.sqrt((Nrtp[f,0,0]-Nrtp[f,1,0])**2 + (Nrtp[f,2,0]-Nrtp[f,3,0])**2))
    
print('Npol flights - average return to points -- ', 
      np.median(np.sqrt((Nrtp[:,0,0]-Nrtp[:,1,0])**2 + (Nrtp[:,2,0]-Nrtp[:,3,0])**2)))

r2pdif_pf = np.zeros([F,4])
print('Flight  X-Flight 618   Y - Flight 618')
for f in np.arange(1,NF):
    r2pdif_pf[f,0] = 0.5*(Nrtp[f,0,0]+Nrtp[f,1,0]) - 0.5*(Nrtp[0,0,0]+Nrtp[0,1,0]) # x
    r2pdif_pf[f,1] = 0.5*(Nrtp[f,2,0]+Nrtp[f,3,0]) - 0.5*(Nrtp[0,2,0]+Nrtp[0,3,0]) # y
    r2pdif_pf[f,2] = np.sqrt( r2pdif_pf[f,0]**2 + r2pdif_pf[f,1]**2 ) # r
    r2pdif_pf[f,3] = np.sqrt(Nrtp[f,0,1]**2 + Nrtp[f,1,1]**2 + Nrtp[f,2,1]**2 + Nrtp[f,3,1]**2) # std
    
print('Flight  avgR stdR relative to flight 618 ---- Npolflights')
for f in np.arange(0,len(Npolflights)):
    print(Npolflights[f],r2pdif_pf[f,2],r2pdif_pf[f,3])
    
#Epolflight order
tlbsa = [0,0,0,0]
tubsa = [100,200,100,500]
tlba = [4930,5370,5300,5810]
tuba = [-1,-1,-1,-1]

Nrtp = np.zeros([len(Epolflights),4,2]) # flight, startX/endY/startY/endY, med/std

for f,fly in enumerate(Epolflights):
    dronedir='/hirax/all_drone_data/datcon_csv/'
    dronedat=drone.Drone_Data(Drone_Directory=dronedir,FLYTAG='FLY'+fly+'.csv',site_class=gbosite)
    
    Nrtp[f,0,0] = np.median(dronedat.coords_xyz_LC[tlbsa[f]:tubsa[f],0]) # start X
    Nrtp[f,1,0] = np.median(dronedat.coords_xyz_LC[tlba[f]:tuba[f],0])  # end X
    Nrtp[f,2,0] = np.median(dronedat.coords_xyz_LC[tlbsa[f]:tubsa[f],1]) # start Y
    Nrtp[f,3,0] = np.median(dronedat.coords_xyz_LC[tlba[f]:tuba[f],1]) #end Y

    Nrtp[f,0,1] = np.std(dronedat.coords_xyz_LC[tlbsa[f]:tubsa[f],0])
    Nrtp[f,1,1] = np.std(dronedat.coords_xyz_LC[tlba[f]:tuba[f],0])
    Nrtp[f,2,1] = np.std(dronedat.coords_xyz_LC[tlbsa[f]:tubsa[f],1])
    Nrtp[f,3,1] = np.std(dronedat.coords_xyz_LC[tlba[f]:tuba[f],1])

print('Flight  delX    delY  delR relative to flight 620 --- Epolflights')
for f in np.arange(len(Epolflights)):
    print(Epolflights[f],(Nrtp[f,0,0]-Nrtp[f,1,0]),Nrtp[f,2,0]-Nrtp[f,3,0],
          np.sqrt((Nrtp[f,0,0]-Nrtp[f,1,0])**2 + (Nrtp[f,2,0]-Nrtp[f,3,0])**2))

print('Epol flights - average return to points -- ', 
      np.median(np.sqrt((Nrtp[:,0,0]-Nrtp[:,1,0])**2 + (Nrtp[:,2,0]-Nrtp[:,3,0])**2)))
    
for f in np.arange(1,NE):
    r2pdif_pf[NF+f,0] = 0.5*(Nrtp[f,0,0]+Nrtp[f,1,0]) - 0.5*(Nrtp[0,0,0]+Nrtp[0,1,0]) # x
    r2pdif_pf[NF+f,1] = 0.5*(Nrtp[f,2,0]+Nrtp[f,3,0]) - 0.5*(Nrtp[0,2,0]+Nrtp[0,3,0]) # y
    r2pdif_pf[NF+f,2] = np.sqrt( r2pdif_pf[f,0]**2 + r2pdif_pf[f,1]**2 ) # r
    r2pdif_pf[NF+f,3] = np.sqrt(Nrtp[f,0,1]**2 + Nrtp[f,1,1]**2 + Nrtp[f,2,1]**2 + Nrtp[f,3,1]**2) # std
    
print('avgR stdR relative to ref flight -- all flights')
print(np.nanmedian(r2pdif_pf[nonzero,2]),np.nanmedian(r2pdif_pf[nonzero,3]))
    
plt.plot(refdif_pf[:,0],r2pdif_pf[:,0],'b.')
plt.plot(np.arange(-5,5),np.arange(-5,5),color='black')
plt.xlabel('Gaussian fit offsets from control flight')
plt.ylabel('return-to-point offsets from control flight')
plt.xlim(-3,3)
plt.ylim(-8,3)

plt.title('X offset')
plt.show()
plt.plot(refdif_pf[:,1],r2pdif_pf[:,1],'b.')
plt.plot(np.arange(-5,5),np.arange(-5,5),color='black')
plt.xlabel('Gaussian fit offsets from control flight')
plt.ylabel('return-to-point offsets from control flight')
plt.title('Y offset')
plt.xlim(-2,3)
plt.ylim(-5,4)
plt.show()


######################################################################
######################################################################
########## Average height and angles during flights ##########
######################################################################
######################################################################

def get_droneavgs(datas):
    dstat = [np.median(datas),np.std(datas),np.max(datas),np.min(datas)] # med/std/max/min
    return dstat

def get_unwrap(angles):
    highs = np.where(angles >180)
    angles[highs] = angles[highs]-360
    return angles



with open('/hirax/GBO_Analysis_Outputs/GBO_flights_forscripts.yaml', 'r') as fff:
    documents = yaml.safe_load(fff)
flights = documents["flight_info"]["flights"]

ddat = np.zeros([4,4,len(flights)]) # height/yaw/pitch/roll; med/std/max/min


for fl,fly in enumerate(flights):
    
    for fi in range(0,len(documents["flight_info"]["flights"])):
        if fly == flights[fi]: f = fi
    print(f, 'FLIGHT: ', documents["flight_info"]["flights"][f])

    tub = documents["flight_info"]["tubs"][f]
    tlb = documents["flight_info"]["tlbs"][f]
    cdat = documents["flight_info"]["cdats"][f]
    tclb = documents["flight_info"]["tclbs"][f]
    tcub = documents["flight_info"]["tcubs"][f]
    fmax = documents["flight_info"]["fmaxes"][f]
    pol = documents["flight_info"]["pols"][f]


    if int(fly) < 537:
        mdir='/hirax/GBO_Aug_2021/TONE_ACQ/'+cdat+'_yale_drone/corr/' # August 2021
        gaindir='/hirax/GBO_Aug_2021/TONE_ACQ/digital_gains/'+cdat+'_yale_drone_yale_digitalgain/' # August 2021
    else:
        mdir='/hirax/GBO_Oct_2021/TONE_ACQ/'+cdat+'_yale_drone/corr/' # Oct 2021
        gaindir='/hirax/GBO_Oct_2021/TONE_ACQ/digital_gains/'+cdat+'_yale_drone_yale_digitalgain/' # Oct 2021

    sdir = os.listdir(mdir)[0]
    datadir=mdir+sdir+'/'
    print(sdir, datadir,gaindir)
    
    dronetest0825=drone.Drone_Data(Drone_Directory=dronedir,FLYTAG='FLY'+fly+'.csv',site_class=gbosite,tlb=tlb,tub=tub)

    print('DONE reading in drone data')
    print(dronetest0825.t_arr_datetime[0], dronetest0825.t_arr_datetime[-1])


  
    if pol=='N' and fly!='623':
        yaws = dronetest0825.yaw
        yaw_uw = get_unwrap(yaws) 
    
        ddat[0,:,fl] = get_droneavgs(dronetest0825.coords_xyz_LC[:,2])#height
        ddat[1,:,fl] = get_droneavgs(yaw_uw)#yaw
        ddat[2,:,fl] = get_droneavgs(dronetest0825.pitch)#pitch
        ddat[3,:,fl] = get_droneavgs(dronetest0825.roll)#roll
    else: 
        ddat[0,:,fl] = get_droneavgs(dronetest0825.coords_xyz_LC[:,2])#height
        ddat[1,:,fl] = get_droneavgs(dronetest0825.yaw)#yaw
        ddat[2,:,fl] = get_droneavgs(dronetest0825.pitch)#pitch
        ddat[3,:,fl] = get_droneavgs(dronetest0825.roll)#roll
    print('avg yaw : ', fly, ddat[1,0,fl])
        
        
# Heights
plt.errorbar(np.arange(0,len(flights)),ddat[0,0,:],yerr=ddat[0,1,:])
plt.ylabel('Height')
plt.show()
print('Average height, and average std of height: ',np.mean(ddat[0,0,:]),np.mean(ddat[0,1,:]))

# Yaw
plt.errorbar(np.arange(0,len(flights)),ddat[1,0,:],yerr=ddat[1,1,:])
plt.ylabel('Yaw')
plt.show()
print('Average yaw, and average std of yaw: ',np.mean(ddat[1,0,:]),np.mean(ddat[1,1,:]))


# Pitch
plt.errorbar(np.arange(0,len(flights)),ddat[2,0,:],yerr=ddat[2,1,:])
plt.ylabel('Pitch')
plt.show()
print('Average pitch, and average std of pitch: ',np.mean(ddat[2,0,:]),np.mean(ddat[2,1,:]))


# Roll
plt.errorbar(np.arange(0,len(flights)),ddat[3,0,:],yerr=ddat[3,1,:])
plt.ylabel('Roll')
plt.show()

print('Average roll, and average std of roll: ',np.mean(ddat[3,0,:]),np.mean(ddat[3,1,:]))

## Estimate the impact of the height on intensity during flight:
A = 1.0
S1 = A / (177 - 2.2)**2
S2 = A / (177 + 2.2)**2

print(S1/S2) # 5\% correction


## Estimate the impact of the yaw angle on intensity during flight:
print(A*np.cos(np.radians(0.6))) # not worth correcting

In [ ]:
# How off in angles am I if I don't correct for this height?

xes = np.arange(-100,100)
heights = 174*np.ones(len(xes))
langles = np.degrees(np.arctan2(xes,heights))

plt.plot(xes,np.abs(langles),'b.',label='angle if height 174m')

heights = 177*np.ones(len(xes))
uangles = np.degrees(np.arctan2(xes,heights))
plt.plot(xes,np.abs(uangles),'r.',label='angle if height is 180m')
plt.xlabel('Distance from dish [m]')
plt.ylabel('Angle [degrees]')
plt.legend()
plt.show()

plt.plot(xes,np.abs(uangles-langles))
plt.xlabel('Distance from dish [m]')
plt.ylabel('180m angles - 174m angles [degrees]')
plt.show()

# back out how different that would have to make the xes themselves, to compare

xesnew = 174*np.tan(np.radians(uangles))
plt.plot(xes,xesnew-xes,'b.')
plt.title('Translate that into an distance on the ground')
plt.xlabel('Actual Distance from dish [m]')
plt.ylabel('Distance I would say from angle [m]')
plt.ylim(-4,4)
plt.show()

In [ ]:
####################################################################################
############################# SECTION 2.2 ##########################################
############################# SECTION 2.2 ##########################################
####################################################################################

######### How bright is the source

def enr2t(enr,h):
    t = 290*(1 + 10**(enr/10.))
    return (3**2 / (2 * h**2))*t


def enr2dbmpH(enr):
    return -174 + enr # returns dBm/Hz

def enr2Jy(enr,h):
    dd = enr2dbmpH(enr) # dbm/Hz
    wph = 1E-3 * 10**(dd/10.) # w/hz
    ww = wph/(2*np.pi*(h**2)) # watts/hz/m^2
    jy = 1E26 * ww # jy
    return jy
 
cables = -3
filters = -0.5
aaron = -0.5
swit = -1
base = 95+cables+filters+aaron+swit
h = 177


# brightest signal
ee = base-36
print('Brightest: ',ee, enr2Jy(ee,h), enr2t(ee,h))

# dimmest signal
ee = base-46
print('Dimmest: ', ee, enr2Jy(ee,h), enr2t(ee,h))

# brighest off 
ee = base-36-40
print('Brightest off: ',ee, enr2Jy(ee,h),enr2t(ee,h))

# dimmest off
ee = base-46-40
print('Dimmest off: ', ee, enr2Jy(ee,h),enr2t(ee,h))


# SEFD for TONE dishes from autocorrelations (https://arxiv.org/pdf/2304.10534.pdf)
# Approx 20kJy in our region of interest


######### How much did the levels change between flights

##################################################
############# Spec an measurements ###############
##################################################


print('46dB flights')
fl46 = np.zeros([5,575])
diffs = np.zeros([5,575])

# 46dB attenuated flights
sdire = '/hirax/GBO_Oct_2021/spec_an_data/FSH4/gbo-1020/'
mess = ['ns-015.csv','ns-016.csv','ns-017.csv','ns-018.csv','ns-019.csv']
for m,ms in enumerate(mess):
    dat = np.loadtxt(sdire+ms,skiprows=100,delimiter=',',usecols=(0,1))
    fl46[m,:] = dat[:,1]
for m,ms in enumerate(mess):
    diffs[m,:] = fl46[m,:] - np.median(fl46,axis=0)
    print(ms, np.mean(diffs[m,:]))
    
 
print('36dB flights')
fl46 = np.zeros([3,575])
diffs = np.zeros([3,575])

sdire = '/hirax/GBO_Oct_2021/spec_an_data/FSH4/gbo-1020/'
mess = ['ns-027.csv','ns-028.csv','ns-029.csv']
for m,ms in enumerate(mess):
    dat = np.loadtxt(sdire+ms,skiprows=100,delimiter=',',usecols=(0,1))
    fl46[m,:] = dat[:,1]
for m,ms in enumerate(mess):
    diffs[m,:] = fl46[m,:] - np.median(fl46,axis=0)
    print(ms, np.mean(diffs[m,:]))

## most within <0.1dB, a few excursions that were ~0.3dB
# convert to percent..
def get_perc(lev,off):
    W0 = 10**(lev/10.)
    W1 = 10**((lev+off)/10)
    return 100*2*(W1-W0)/(W0+W1)

print(get_perc(-60,-0.3))

print(get_perc(-54,-0.1))


In [ ]:
####################################################################################
############################# SECTION 2.3 ##########################################
############################# SECTION 2.3 ##########################################
####################################################################################

## Timing correction for table ##

ymlfiles = os.listdir(ymldir)
flytags = []
toffs = []
for ymlfile in ymlfiles:
    with open(ymldir+ymlfile, 'r') as fff:
        documents = yaml.safe_load(fff)
        print(documents["file_info"]["FLYTAG"],documents["timing_params"]["t_delta_dji"])
        flytags.append(documents["file_info"]["FLYTAG"])
        toffs.append(documents["timing_params"]["t_delta_dji"])
labels = flytags
x = np.arange(0,len(toffs))
# Plot both separately

plt.plot(toffs,'k.')
plt.xticks(x, labels, rotation='vertical')
plt.ylabel('Timing offset (s)')
plt.show()




## Flight info plot ##

Npolflights = ['618','619','623','625','646','647','533','536']
Epolflights = ['620','648','649','535']

pflights=Npolflights
find=900

Nflightarr = np.zeros([64,64,16,len(Npolflights)]) # pixel, pixel, freq, dish, Nflights
normarr = np.zeros([64,64,16,len(Npolflights)])

for i, fly in enumerate(Npolflights):

    pol, pols, cpols, attn, fi = get_flightinfo(fly)
    print(fly,pol,pols,cpols, attn)
    concytest=[glob.glob(pckldir+'*'+fly+'*')[0] for x in flights]
    print(concytest[fi])

    with open(glob.glob(pckldir+'*'+fly+'*')[0], 'rb') as pfile:
        print(pfile)
        concattest1=pickle.load(pfile)
    t_cut=concattest1.inds_on

    beam=bp.Beammap(concatlist=pcklarr[[fi]],gfitlist=gfitarr[[fi]],coordsys='cartesian',
                 normalization='Gauss',operation='coadd',d0args=[-80,80,2.5],
                 d1args=[-80,80,2.5],Fargs=[find,find+1,1],f_index=find,vplot=False)

    #if fly == '618':
    #    normarr[:,:,:,i] = 1.0
    #elif fly == '620':
    #    normarr[:,:,:,i] = 1.0
    if fly in Npolflights:
        pklfile = ampdir+'FLY'+str(fly)+'_Corrected_amplitudes.pkl'
        with open(pklfile, 'rb') as inp:
            amps = pickle.load(inp)
        normarr[:,:,:,i] = amps[0,find,:]
    elif fly in Epolflights:
        pklfile = ampdir+'FLY'+str(fly)+'_Corrected_amplitudes.pkl'
        with open(pklfile, 'rb') as inp:
            amps = pickle.load(inp)
        normarr[:,:,:,i] = amps[0,find,:]
    Nflightarr[:,:,:,i] = beam.V_LC_mean[:,:,0,:,0]*normarr[:,:,:,i]
    
Eflightarr = np.zeros([64,64,16,len(Epolflights)]) # pixel, pixel, freq, dish, Nflights
normarr = np.zeros([64,64,16,len(Epolflights)])

for i, fly in enumerate(Epolflights):

    pol, pols, cpols, attn, fi = get_flightinfo(fly)
    print(fly,pol,pols,cpols, attn)
    concytest=[glob.glob(pckldir+'*'+fly+'*')[0] for x in flights]
    print(concytest[fi])

    with open(glob.glob(pckldir+'*'+fly+'*')[0], 'rb') as pfile:
        print(pfile)
        concattest1=pickle.load(pfile)
    t_cut=concattest1.inds_on

    beam=bp.Beammap(concatlist=pcklarr[[fi]],gfitlist=gfitarr[[fi]],coordsys='cartesian',
                 normalization='Gauss',operation='coadd',d0args=[-80,80,2.5],
                 d1args=[-80,80,2.5],Fargs=[find,find+1,1],f_index=find,vplot=False)

    #if fly == '618':
    #    normarr[:,:,:,i] = 1.0
    #elif fly == '620':
    #    normarr[:,:,:,i] = 1.0
    if fly in Npolflights:
        pklfile = ampdir+'FLY'+str(fly)+'_Corrected_amplitudes.pkl'
        with open(pklfile, 'rb') as inp:
            amps = pickle.load(inp)
        normarr[:,:,:,i] = amps[0,find,:]
    elif fly in Epolflights:
        pklfile = ampdir+'FLY'+str(fly)+'_Corrected_amplitudes.pkl'
        with open(pklfile, 'rb') as inp:
            amps = pickle.load(inp)
        normarr[:,:,:,i] = amps[0,find,:]
    Eflightarr[:,:,:,i] = beam.V_LC_mean[:,:,0,:,0]*normarr[:,:,:,i]
    
fig,axes = plt.subplots(2, 6, sharex=True,sharey=True,figsize=(16,8),squeeze=True)


# Npol flights
chind = N_pols[0]
for p,pp in enumerate(Npolflights):
    if p<6: 
        i=0
        pi = p
    else: 
        i=1
        pi=p-6
    axes[i,pi].pcolormesh(beam.d0_centers_grid[:,:,0],beam.d1_centers_grid[:,:,0],
               np.abs(Nflightarr[:,:,chind,p]),cmap=ccm,vmin=1E-3,vmax=1.0,norm=LogNorm())#,edgecolors='k')
    axes[i,pi].set_xlim(-100,100)
    axes[i,pi].set_ylim(-100,100)
    axes[i,pi].text(-90,85,'Flight '+str(pp)+' (N-pol)')
    if p==0 or p==6:
        axes[i,pi].set_ylabel('Distance [m]')
    if p>5:
        axes[i,pi].set_xlabel('Distance [m]')

# Epol flights
chind = E_pols[0]
for p,pp in enumerate(Epolflights):
    axes[1,pi+p+1].pcolormesh(beam.d0_centers_grid[:,:,0],beam.d1_centers_grid[:,:,0],
               np.abs(Eflightarr[:,:,chind,p]),cmap=ccm,vmin=1E-3,vmax=1.0,norm=LogNorm())#,edgecolors='k')
    axes[1,pi+p+1].text(-90,85,'Flight '+str(pp)+' (E-pol)')
    axes[1,pi+p+1].set_xlim(-100,100)
    axes[1,pi+p+1].set_ylim(-100,100)
    axes[i,pi+p+1].set_xlabel('Distance [m]')
#plt.colorbar()

plt.show()

In [ ]:
####################################################################################
############################# SECTION 2.4 ##########################################
############################# SECTION 2.4 ##########################################
####################################################################################

freq_t = good_freqs[4]
markers = ['s','v','p','^','o','>','P','x','+']
F = len(Npolflights)+len(Epolflights)
colorz=cm_to_discrete(ccm,F)

## North-polarized flights:
#pols = N_pols
#fig = plt.figure(figsize=(8,8))
#for f,fly in enumerate(Npolflights):
#    ffile = glob.glob(fitdir+'*'+fly+'*')[0] #looping through flights and finding files with data for them
#    fits = np.load(ffile) #actually loads file
#    for p,pol in enumerate(pols):
#        plt.plot(fits['G_popt'][pol,freq_t,1],fits['G_popt'][pol,freq_t,3],
#                 marker=markers[p],color=colorz[f],linestyle='None',label=fly) #plotting x centroid (1) at certain frequency and y centroid (3) at certain frequency 
#plt.xlabel('X centroid [m]')
#plt.ylabel('Y centroid [m]')
#plt.title('North Polarized Flights')
#plt.show()

## East-polarized flights:
#pols = E_pols
#fig = plt.figure(figsize=(8,8))
#for f,fly in enumerate(Epolflights):
#    ffile = glob.glob(fitdir+'*'+fly+'*')[0]
#    fits = np.load(ffile)
#    for p,pol in enumerate(pols):
#        try:
#            plt.plot(fits['G_popt'][pol,freq_t,1],fits['G_popt'][pol,freq_t,3],
#                     color=colorz[f+len(Npolflights)],marker=markers[p],linestyle='None',label=fly)
#        except:''
#plt.xlabel('X centroid [m]')
#plt.ylabel('Y centroid [m]')
#plt.title('East Polarized Flight')
#plt.show()

## ## Take a median over lower half of frequencies ## ##
# East and North flights together
#Red X where Wills code thinks dish was

cms = np.zeros([F,len(N_pols),2]) # flight, pol (0-7), X,Y

pols = N_pols
for f,fly in enumerate(Npolflights):
    fits = np.load(glob.glob(fitdir+'*'+fly+'*')[0])
    for p,pol in enumerate(pols):
        cms[f,p,0] = np.nanmedian(fits['G_popt'][pol,650:-1,1]) # less than 550 MHz
        cms[f,p,1] = np.nanmedian(fits['G_popt'][pol,650:-1,3]) # less than 550 MHz
        
hf = f
pols = E_pols
for f,fly in enumerate(Epolflights):
    fits = np.load(glob.glob(fitdir+'*'+fly+'*')[0])
    for p,pol in enumerate(pols):
        try:
            cms[f+hf+1,p,0] = np.nanmedian(fits['G_popt'][pol,650:-1,1]) # less than 550 MHz
            cms[f+hf+1,p,1] = np.nanmedian(fits['G_popt'][pol,650:-1,3]) # less than 550 MHz
        except:''



fig = plt.figure(figsize=(10,10))
for f in range(0,F):
    for p in range(0,len(pols)):
        plt.scatter(cms[f,p,0],cms[f,p,1],c=colorz[f],s=100)
for p in range(0,len(cms[0,:,0])):
    plt.plot(np.nanmedian(cms[:,p,0]),np.nanmedian(cms[:,p,1]),color='black',marker='P',markersize=15)
    
    plt.text(np.median(cms[:,p,0])-3,np.median(cms[:,p,1])-4,'inputs: '+str(E_pols[p])+'+'+str(N_pols[p]),fontsize=15)

    R = 3
    theta=np.arange(0,2*np.pi,0.001)
    circx,circy = [R*np.cos(theta),R*np.sin(theta)]
    plt.plot(np.median(cms[:,p,0]) + circx, np.median(cms[:,p,1])+circy,color='black')

dishes = ['Dish 0','Dish 7','Dish 2','Dish 3','Dish 4','Dish 5','Dish 6','Dish 8']
for d in range(0,len(gbosite.coords[:,0]),2):
    plt.plot(gbosite.coords[d,0],gbosite.coords[d,1],color='grey',marker='X',markersize=10)
    plt.text(gbosite.coords[d,0]-1.5,gbosite.coords[d,1]+0.5, dishes[int(d/2)], color='grey',fontsize=15,alpha=0.8)

plt.xlabel('X centroid [m]',fontsize=16)
plt.ylabel('Y centroid [m]',fontsize=16)
for f in range(0,F):
    if f < len(Npolflights):
        plt.text(-32,17-1.5*f,'Flight '+str(Npolflights[f]),color=colorz[f],fontsize=20)
    else:
        plt.text(-32,17-1.5*f,'Flight '+str(Epolflights[f-len(Npolflights)]),color=colorz[f],fontsize=20)
plt.text(-5,-12,'+ median',color='black',fontsize=20)
plt.text(-5,-14,'x nominal',color='grey',fontsize=20)
plt.grid()
plt.show()



# ############################# SECTION 3 ############################################
############################# SECTION 3 ############################################
############################# SECTION 3 ############################################
############################# SECTION 3 ############################################
####################################################################################




In [ ]:
####################################################################################
############################# SECTION 3.4 ##########################################
############################# SECTION 3.4 ##########################################
####################################################################################


#### Make a nice plot that shows the slices through the beam after normalization ####


Npolflights = ['618','619','623','625','646','647','533','536']
Epolflights = ['620','648','649','535']

pflights=Npolflights
find=900

flightarr = np.zeros([64,64,16,len(pflights)]) # pixel, pixel, freq, dish, Nflights
normarr = np.zeros([64,64,16,len(pflights)])

for i, fly in enumerate(pflights):

    pol, pols, cpols, attn, fi = get_flightinfo(fly)
    print(fly,pol,pols,cpols, attn)
    concytest=[glob.glob(pckldir+'*'+fly+'*')[0] for x in flights]
    print(concytest[fi])

    with open(glob.glob(pckldir+'*'+fly+'*')[0], 'rb') as pfile:
        print(pfile)
        concattest1=pickle.load(pfile)
    t_cut=concattest1.inds_on

    beam=bp.Beammap(concatlist=pcklarr[[fi]],gfitlist=gfitarr[[fi]],
                 normalization='Gauss',operation='coadd',d0args=[-1*sz,sz,2.5],
                 d1args=[-1*sz,sz,2.5],Fargs=[find,find+1,1],f_index=find,vplot=False)

    if fly == '618':
        normarr[:,:,:,i] = 1.0
    elif fly == '620':
        normarr[:,:,:,i] = 1.0
    elif fly in Npolflights[1::]:
        pklfile = ampdir+'FLY'+str(fly)+'_Corrected_amplitudes.pkl'
        with open(pklfile, 'rb') as inp:
            amps = pickle.load(inp)
        normarr[:,:,:,i] = amps[0,find,:]
    elif fly in Epolflights[1::]:
        pklfile = ampdir+'FLY'+str(fly)+'_Corrected_amplitudes.pkl'
        with open(pklfile, 'rb') as inp:
            amps = pickle.load(inp)
        normarr[:,:,:,i] = amps[0,find,:]
    flightarr[:,:,:,i] = beam.V_LC_mean[:,:,0,:,0]*normarr[:,:,:,i]
    
xx = 0
chind = pols[0]
colx = get_slice(beam.d0_centers_grid[:,:,0],beam.d1_centers_grid[:,:,0],beam.V_LC_mean[:,:,0,chind,0],xx,'v')
coly = get_slice(beam.d0_centers_grid[:,:,0],beam.d1_centers_grid[:,:,0],beam.V_LC_mean[:,:,0,chind,0],xx,'h')
fig = plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
angls = np.degrees(np.arctan(beam.d1_centers_grid[colx,:,0]/177.0))
for i,p in enumerate(pflights):
    plt.semilogy(angls,flightarr[colx,:,chind,i],'o',
             color=colors[i],label='Flight '+str(p))
#plt.semilogy(beam_coad.y_centers_grid[colx,:,chind],beam_coad.V_LC_operation[colx,:,0,chind],
#             label='Co-add',color='black')

plt.ylim(0.0001,1)
plt.xlim(-25,25)
plt.text(-24,0.3,'Vertical slice @ origin',fontsize=fsz)
plt.legend(loc='lower center')
plt.text(15,0.3,'488.4 MHz',fontsize=fsz)
plt.text(15,0.1,'Input 1',fontsize=fsz)

plt.subplot(2,1,2)
angls = np.degrees(np.arctan(beam.d0_centers_grid[:,coly,0]/177.0))
for i,p in enumerate(pflights):
    plt.semilogy(angls,flightarr[:,coly,chind,i],'o',
             color=colors[i],label='Flight '+str(p))
    plt.ylim(0.0001,1)
    plt.xlim(-25,25)
#plt.semilogy(beam_coad.x_centers_grid[:,coly,0],beam_coad.V_LC_operation[:,coly,0,chind],
#             label='Co-add',color='black')
plt.text(-24,0.3,'Horizontal slice @ origin',fontsize=fsz)
plt.legend(loc='lower center')
plt.xlabel('Angle [degrees]',fontsize=fsz)



####################################################################################
############################# SECTION 4 ############################################
############################# SECTION 4 ############################################
############################# SECTION 4 ############################################
############################# SECTION 4 ############################################
####################################################################################

In [ ]:
## FWHMS for Coadded beam maps, with comparison to Gaussian fit averages ####

coadded = np.load('/hirax/GBO_Analysis_Outputs/Averaged_2dGauss_and_Airy_Param.npz')
print(coadded['averaged'][1,900,2,0],coadded['averaged'][1,900,2,1])

with open('/hirax/GBO_Analysis_Outputs/emcee_fits.pkl', 'rb') as inp:
            emcfile = pickle.load(inp)
        
colorz=pu.cm_to_discrete(ccm, 16)

## Plot just the gaussian fit values and std across the main beam flights ##
fig,axes = plt.subplots(2, 1, sharex=True,sharey=True,figsize=(15,10),squeeze=True)
for i in np.arange(0,16):
    if i in N_pols:
        ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,0])))
        yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,1])))
        axes[0].errorbar(freqs[good_freqs],ys,yerr = yerrs,
                 fmt='None',elinewidth=4,ecolor=colorz[i],alpha = 0.3,label='Input '+str(i))
    if i in E_pols:
        ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,0])))
        axes[0].plot(freqs[good_freqs],ys,marker='o',linestyle='None',
                     color=colorz[i],alpha = 0.3,label='Input '+str(i))

axes[0].set_ylabel ('X FWHM [deg]', fontsize=20)
axes[0].set_ylim(5,8.5)
axes[0].set_xlim(400,600)
axes[0].legend(fontsize = 14, ncol=5, markerscale=1,loc='lower left')
    
for i in np.arange(0,16):
    if i in N_pols:
        ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,0])))
        yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,1])))
        axes[1].errorbar(freqs[good_freqs],ys,yerr = yerrs,
                 fmt='None',ecolor=colorz[i],alpha = 0.3,label='Input '+str(i))
    if i in E_pols:
        ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,0])))
        axes[1].plot(freqs[good_freqs],ys,marker='o',linestyle='None',color=colorz[i],alpha = 0.3,label='Input '+str(i))

axes[1].set_xlabel('Frequency [MHz]', fontsize=20)
axes[1].set_ylabel ('Y FWHM [deg]', fontsize=20)
axes[1].set_ylim(5,8.5)
axes[1].set_xlim(400,600)
axes[1].legend(fontsize = 14, ncol=5, markerscale=1,loc='lower left')
plt.show()


fig,axes = plt.subplots(4, 1, sharex=True,figsize=(15,12),gridspec_kw={'height_ratios': [2,1,2,1]})
for i in np.arange(0,16):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,1]))
    axes[0].errorbar(freqs[good_freqs],mcs,yerr=mcerr,
                elinewidth=3,ecolor=colorz[i],
                linestyle='None',alpha = 0.4,label='Input '+str(i))
    axes[1].errorbar(freqs[good_freqs],mcs-ys,yerr = yerrs,fmt='None',
                ecolor=colorz[i],alpha = 0.3,label='Input '+str(i))
 
axes[0].set_ylabel('[deg]', fontsize=fsz)
axes[0].text(402,8,'E/W FWHM [deg]', fontsize=fsz)
axes[0].set_ylim(5,8.5)
axes[0].set_xlim(400,600)
axes[0].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')
axes[1].set_ylabel('[deg]', fontsize=fsz)
#axes[1].set_ylim(-3,3)
axes[1].text(402,0.6,'Residuals',fontsize=fsz-2)

    
for i in np.arange(0,16):
        ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,0])))
        yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,1])))
        mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,0]))
        mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,1]))
        axes[2].errorbar(freqs[good_freqs],mcs,yerr=mcerr,
                        elinewidth=3,ecolor=colorz[i],
                        linestyle='None',alpha = 0.4,label='Input '+str(i))
        axes[3].errorbar(freqs[good_freqs],ys-mcs,yerr = mcerr,
                 fmt='None',ecolor=colorz[i],alpha = 0.3,label='Input '+str(i))
axes[2].set_ylabel('[deg]', fontsize=fsz)
axes[2].text(402,8,'N/S FWHM [deg]', fontsize=fsz)
axes[2].set_ylim(5,8.5)
axes[3].set_ylabel('[deg]', fontsize=fsz)
axes[3].set_xlabel('Frequency [MHz]', fontsize=fsz)
#axes[3].set_ylim(-3,3)
axes[3].set_xlim(400,600)
axes[3].text(402,0.6,'Residuals',fontsize=fsz-2)
plt.show()

fig,axes = plt.subplots(2, 1, sharex=True,figsize=(15,12))

for i in np.arange(16):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,1]))
    axes[0].errorbar(freqs[good_freqs],mcs,yerr=mcerr,
                elinewidth=4,ecolor=colorz[i],
                linestyle='None',alpha = 0.4,label='Input '+str(i))
 
axes[0].set_ylabel('E/W FWHM [deg]', fontsize=fsz)
#axes[0].text(402,8,'E/W FWHM [deg]', fontsize=fsz)
axes[0].set_ylim(5,8.5)
axes[0].set_xlim(400,600)
axes[0].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')

    
for i in np.arange(16):
        ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,0])))
        yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,1])))
        mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,0]))
        mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,1]))
        axes[1].errorbar(freqs[good_freqs],mcs,yerr=mcerr,
                        elinewidth=4,ecolor=colorz[i],
                        linestyle='None',alpha = 0.4,label='Input '+str(i))
axes[1].set_ylabel('N/S FWHM [deg]', fontsize=fsz)
#axes[1].text(402,8,'N/S FWHM [deg]', fontsize=fsz)
axes[1].set_ylim(5,8.5)
axes[1].set_xlim(400,600)
axes[1].set_xlabel('Frequency [MHz]',fontsize=fsz)
plt.show()


colorz=pu.cm_to_discrete(ccm, 5)
fig,axes = plt.subplots(4, 1, sharex=True,sharey=True,figsize=(15,12))
diffs = []
ms = 0

for p, i in enumerate(N_pols):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,1]))
    axes[0].errorbar(freqs[good_freqs],mcs,yerr=mcerr,
                elinewidth=3,ecolor=colorz[p],
                linestyle='None',alpha = 0.4,label='Input '+str(i))
    diffs.append(np.nanmedian(np.abs(mcs-ys)))
    ms+=1
for p,i in enumerate(E_pols):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,1]))
    axes[1].errorbar(freqs[good_freqs],mcs,yerr=mcerr,
                elinewidth=3,ecolor=colorz[p],
                linestyle='None',alpha = 0.4,label='Input '+str(i))
    
    diffs.append(np.nanmedian(np.abs(mcs-ys)))
    ms+=1
axes[0].set_ylabel('FWHM [deg]', fontsize=fsz)
axes[0].text(402,8,'E/W (N pols)', fontsize=fsz)
axes[0].set_ylim(5,8.5)
axes[0].set_xlim(400,600)
axes[0].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')
axes[1].set_ylabel('FWHM [deg]', fontsize=fsz)
axes[1].text(402,8,'E/W (E pols)',fontsize=fsz)
axes[1].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')

    
for p,i in enumerate(N_pols):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,1])) 
    axes[2].errorbar(freqs[good_freqs],mcs,yerr=mcerr,
                    elinewidth=3,ecolor=colorz[p],
                    linestyle='None',alpha = 0.4,label='Input '+str(i))
    diffs.append(np.nanmedian(np.abs(mcs-ys)))
    ms+=1
for p,i in enumerate(E_pols):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,1])) 
    axes[3].errorbar(freqs[good_freqs],mcs,yerr=mcerr,
                    elinewidth=3,ecolor=colorz[p],
                    linestyle='None',alpha = 0.4,label='Input '+str(i))
    diffs.append(np.nanmedian(np.abs(mcs-ys)))
    ms+=1
axes[2].set_ylabel('FWHM [deg]', fontsize=fsz)
axes[2].text(402,8,'N/S (N pols)', fontsize=fsz)
axes[3].set_ylabel('FWHM [deg]', fontsize=fsz)
axes[3].set_xlabel('Frequency [MHz]', fontsize=fsz)
axes[3].text(402,8,'N/S (E pols)',fontsize=fsz)
axes[2].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')
axes[3].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')
plt.show()
 
print('Average Consistency between Gaussian fits and EMCEE coadded fits: ', np.sum(diffs)/ms)



fig,axes = plt.subplots(4, 1, sharex=True,sharey=True,figsize=(15,12))
for p, i in enumerate(N_pols):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,1]))
    axes[0].plot(freqs[good_freqs],mcs,color=colorz[p],
                linestyle='None',marker='o',alpha = 0.3,label='Input '+str(i))
for p,i in enumerate(E_pols):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,2,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,2,1]))
    axes[2].plot(freqs[good_freqs],mcs,color=colorz[p],
                linestyle='None',marker='o',alpha = 0.3,label='Input '+str(i))
    
axes[0].set_ylabel('FWHM [deg]', fontsize=fsz)
axes[0].text(402,8,'E/W (N pols)', fontsize=fsz)
axes[0].set_ylim(4.5,8.5)
axes[0].set_xlim(400,600)
axes[0].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')
axes[2].set_ylabel('FWHM [deg]', fontsize=fsz)
axes[2].text(402,8,'E/W (E pols)',fontsize=fsz)
axes[2].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')

    
for p,i in enumerate(N_pols):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,1])) 
    axes[1].plot(freqs[good_freqs],mcs,color=colorz[p],
                    linestyle='None',marker='o',alpha = 0.3,label='Input '+str(i))
for p,i in enumerate(E_pols):
    ys = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,0])))
    yerrs = convert_sig_to_FWHM((convert_to_angle(coadded['averaged'][i,good_freqs,4,1])))
    mcs = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,0]))
    mcerr = convert_sig_to_FWHM(convert_to_angle(emcfile[good_freqs,i,4,1])) 
    axes[3].plot(freqs[good_freqs],mcs,color=colorz[p],
                    linestyle='None',marker='o',alpha = 0.3,label='Input '+str(i))
axes[1].set_ylabel('FWHM [deg]', fontsize=fsz)
axes[1].text(402,8,'N/S (N pols)', fontsize=fsz)
axes[3].set_ylabel('FWHM [deg]', fontsize=fsz)
axes[3].set_xlabel('Frequency [MHz]', fontsize=fsz)
axes[3].text(402,8,'N/S (E pols)',fontsize=fsz)
axes[1].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')
axes[3].legend(fontsize = 12, ncol=8, markerscale=1,loc='lower left')
plt.show()




In [ ]:
## Slice plots of co-added beams, including error bars ####

colorz = pu.cm_to_discrete(ccm, 5)

# set which pol:
copoldir = 'N'
pols = N_pols
chind = pols[0]


picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)

chind = pols[0]
col1,col2 = get_polar_slice(ptest.d0_centers_grid[:,:,chind],
                          ptest.d1_centers_grid[:,:,chind],0.0)

col1_90,col2_90 = get_polar_slice(ptest.d0_centers_grid[:,:,chind],
                          ptest.d1_centers_grid[:,:,chind],np.pi/2.)


fig,axs = plt.subplots(2, 2, sharex=True,sharey=True,figsize=(10,8))
for i,chind in enumerate(N_pols):  
    axs[0,0].errorbar(np.degrees(ptest.d1_centers_grid[col1,:,chind]),
                    ptest.V_LC_operation[col1,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1,:,0,chind],marker='o',
             color=colorz[i],label='Input :'+str(chind))
    axs[0,0].errorbar(-1.0*np.degrees(ptest.d1_centers_grid[col2,:,chind]),
                    ptest.V_LC_operation[col2,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1,:,0,chind],marker='o',
             color=colorz[i])
    axs[0,1].errorbar(np.degrees(ptest.d1_centers_grid[col1_90,:,chind]),
                    ptest.V_LC_operation[col1_90,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1_90,:,0,chind],marker='o',
             color=colorz[i],label='Input :'+str(chind))
    axs[0,1].errorbar(-1.0*np.degrees(ptest.d1_centers_grid[col2_90,:,chind]),
                    ptest.V_LC_operation[col2_90,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1_90,:,0,chind],marker='o',
             color=colorz[i])
    

copoldir = 'E'
pols = E_pols
chind = pols[0]
picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)
    
for i,chind in enumerate(E_pols):
    axs[1,0].errorbar(np.degrees(ptest.d1_centers_grid[col1,:,chind]),
                    ptest.V_LC_operation[col1,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1,:,0,chind], marker='o',
                    color=colorz[i],label='Input :'+str(chind))
    axs[1,0].errorbar(-1.0*np.degrees(ptest.d1_centers_grid[col2,:,chind]),
                    ptest.V_LC_operation[col2,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1,:,0,chind],marker='o',
                     color=colorz[i])
    axs[1,1].errorbar(np.degrees(ptest.d1_centers_grid[col1_90,:,chind]),
                    ptest.V_LC_operation[col1_90,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1_90,:,0,chind],marker='o',
                    color=colorz[i],label='Input :'+str(chind))
    axs[1,1].errorbar(-1.0*np.degrees(ptest.d1_centers_grid[col2_90,:,chind]),
                    ptest.V_LC_operation[col2_90,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1_90,:,0,chind],marker='o',
                     color=colorz[i])
    
axs[1,0].set_xlabel('Angle [degrees]')
axs[1,1].set_xlabel('Angle [degrees]')
axs[0,0].set_ylabel('Amplitude')
axs[1,0].set_ylabel('Amplitude')
axs[0,0].legend(loc='lower center')
axs[1,0].legend(loc='lower center')
axs[0,1].legend(loc='lower center')
axs[1,1].legend(loc='lower center')
axs[0,0].text(-25,0.5,'theta=0',fontsize=14)
axs[0,1].text(-25,0.5,'theta=0',fontsize=14)
axs[1,0].text(-25,0.5,'theta=90',fontsize=14)
axs[1,1].text(-25,0.5,'theta=90',fontsize=14)
axs[0,0].set_xlim(-30,30)
axs[0,0].set_ylim(1E-4,1)
plt.show()



# set which pol:
copoldir = 'N'
pols = N_pols
chind = pols[0]

picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)

chind = pols[0]
col1,col2 = get_polar_slice(ptest.d0_centers_grid[:,:,chind],
                          ptest.d1_centers_grid[:,:,chind],0.0)

col1_90,col2_90 = get_polar_slice(ptest.d0_centers_grid[:,:,chind],
                          ptest.d1_centers_grid[:,:,chind],np.pi/2.)


fig,axs = plt.subplots(2, 2, sharex=True,sharey=True,figsize=(10,8))
for i,chind in enumerate(N_pols):  
    axs[0,0].errorbar(np.degrees(ptest.d1_centers_grid[col1,:,chind]),
                    ptest.V_LC_operation[col1,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1,:,0,chind],marker='o',
             color=colorz[i],label='Input :'+str(chind))
    axs[0,0].errorbar(-1.0*np.degrees(ptest.d1_centers_grid[col2,:,chind]),
                    ptest.V_LC_operation[col2,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1,:,0,chind],marker='o',
             color=colorz[i])
    axs[0,1].errorbar(np.degrees(ptest.d1_centers_grid[col1_90,:,chind]),
                    ptest.V_LC_operation[col1_90,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1_90,:,0,chind],marker='o',
             color=colorz[i],label='Input :'+str(chind))
    axs[0,1].errorbar(-1.0*np.degrees(ptest.d1_centers_grid[col2_90,:,chind]),
                    ptest.V_LC_operation[col2_90,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1_90,:,0,chind],marker='o',
             color=colorz[i])
    

copoldir = 'E'
pols = E_pols
chind = pols[0]
picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)
    
for i,chind in enumerate(E_pols):
    axs[1,0].errorbar(np.degrees(ptest.d1_centers_grid[col1,:,chind]),
                    ptest.V_LC_operation[col1,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1,:,0,chind], marker='o',
                    color=colorz[i],label='Input :'+str(chind))
    axs[1,0].errorbar(-1.0*np.degrees(ptest.d1_centers_grid[col2,:,chind]),
                    ptest.V_LC_operation[col2,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1,:,0,chind],marker='o',
                     color=colorz[i])
    axs[1,1].errorbar(np.degrees(ptest.d1_centers_grid[col1_90,:,chind]),
                    ptest.V_LC_operation[col1_90,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1_90,:,0,chind],marker='o',
                    color=colorz[i],label='Input :'+str(chind))
    axs[1,1].errorbar(-1.0*np.degrees(ptest.d1_centers_grid[col2_90,:,chind]),
                    ptest.V_LC_operation[col2_90,:,0,chind],
                    yerr=ptest.V_LC_operation_err[col1_90,:,0,chind],marker='o',
                     color=colorz[i])
    
axs[1,0].set_xlabel('Angle [degrees]')
axs[1,1].set_xlabel('Angle [degrees]')
axs[0,0].set_ylabel('Amplitude')
axs[1,0].set_ylabel('Amplitude')
axs[0,0].legend(loc='lower center')
axs[1,0].legend(loc='lower center')
axs[0,1].legend(loc='lower center')
axs[1,1].legend(loc='lower center')
axs[0,0].text(-25,0.5,'theta=0',fontsize=14)
axs[0,1].text(-25,0.5,'theta=0',fontsize=14)
axs[1,0].text(-25,0.5,'theta=90',fontsize=14)
axs[1,1].text(-25,0.5,'theta=90',fontsize=14)
axs[0,0].set_yscale('log')
axs[0,0].set_xlim(-30,30)
axs[0,0].set_ylim(1E-4,1)
plt.show()

In [ ]:
### Polar beam maps at freqind 900, maps and errors ###

fig, axs = plt.subplots(2, 5, subplot_kw=dict(projection="polar"),figsize=(16,8))

# N pols first
copoldir = 'N'
pols = N_pols
find = 900

picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)


for i,chind in enumerate(N_pols):
    axs[0, i].pcolormesh(ptest.d0_edges_grid[:,:,chind],ptest.d1_edges_grid[:,:,chind],
                10*np.log10(ptest.V_LC_operation[:,:,0,chind]),cmap=ccm,vmin=-50,vmax=0)
    axs[0, i].set_title('Input: '+str(chind),fontsize=fsz)

copoldir = 'E'
pols = E_pols

picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)


for i,chind in enumerate(E_pols):
    im = axs[1, i].pcolormesh(ptest.d0_edges_grid[:,:,chind],ptest.d1_edges_grid[:,:,chind],
                10*np.log10(ptest.V_LC_operation[:,:,0,chind]),cmap=ccm,vmin=-50,vmax=0)
    axs[1, i].set_title('Input: '+str(chind),fontsize=fsz)
    #axs[1, i].colorbar()


cbar=fig.colorbar(im,ax=axs,aspect=50)
cbar.set_label('Amplitude [dB]',fontsize=fsz)
plt.show()

### Polar beam map - errors at freqind 900 ###

fig, axs = plt.subplots(2, 5, subplot_kw=dict(projection="polar"),figsize=(16,8))

# N pols first
copoldir = 'N'
pols = N_pols
find = 900

picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)


for i,chind in enumerate(N_pols):
    axs[0, i].pcolormesh(ptest.d0_edges_grid[:,:,chind],ptest.d1_edges_grid[:,:,chind],
                10*np.log10(ptest.V_LC_operation_err[:,:,0,chind]),cmap=ccm,vmin=-50,vmax=0)
    axs[0, i].set_title('Input: '+str(chind),fontsize=fsz)

copoldir = 'E'
pols = E_pols

picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)


for i,chind in enumerate(E_pols):
    im = axs[1, i].pcolormesh(ptest.d0_edges_grid[:,:,chind],ptest.d1_edges_grid[:,:,chind],
                10*np.log10(ptest.V_LC_operation_err[:,:,0,chind]),cmap=ccm,vmin=-50,vmax=0)
    axs[1, i].set_title('Input: '+str(chind),fontsize=fsz)
    #axs[1, i].colorbar()


cbar=fig.colorbar(im,ax=axs,aspect=50)
cbar.set_label('Variance [dB]',fontsize=fsz)
plt.show()

## Plot total counts ###

fig, axs = plt.subplots(1,2, subplot_kw=dict(projection="polar"),figsize=(16,8))

# N pols first
copoldir = 'N'
pols = N_pols
find = 900

picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)


chind = pols[0]
axs[0].pcolormesh(ptest.d0_edges_grid[:,:,chind],ptest.d1_edges_grid[:,:,chind],
            ptest.V_LC_operation_count[:,:,0,chind],cmap=ccm,vmin=0,vmax=40)
axs[0].set_title('N-pol flights',fontsize=fsz)
copoldir = 'E'
pols = E_pols

picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
with open(picklefile, 'rb') as inp:
    ptest = pickle.load(inp)


chind = pols[0]
im = axs[1].pcolormesh(ptest.d0_edges_grid[:,:,chind],ptest.d1_edges_grid[:,:,chind],
            ptest.V_LC_operation_count[:,:,0,chind],cmap=ccm,vmin=0,vmax=40)
axs[1].set_title('E-pol flights',fontsize=fsz)
cbar=fig.colorbar(im,ax=axs,aspect=50)
cbar.set_label('Counts',fontsize=fsz)

fig.show()


In [ ]:
### slice vs frequency ### 

fig, axs = plt.subplots(2, 3,figsize=(16,8),sharex=True,sharey=True)
whichang = [0.0, np.pi/4.0, np.pi/2.0]

# set which pol:
copoldir = 'N'
pols = N_pols
chind = pols[0]

for find in good_freqs:
    picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
    with open(picklefile, 'rb') as inp:
        ptest = pickle.load(inp)
    f = freqs[find]
    for a,arad in enumerate(whichang):
        col1,col2 = get_polar_slice(ptest.d0_centers_grid[:,:,chind],
                          ptest.d1_centers_grid[:,:,chind],arad)
        axs[0,a].scatter(np.degrees(ptest.d1_centers_grid[col1,:,chind]),
                f*np.ones(len(ptest.d1_centers_grid[col1,:,chind])),
                c=10*np.log10(abs(ptest.V_LC_operation[col1,:,0,chind])),vmin=-40,vmax=0,cmap=ccm)
        axs[0,a].scatter(np.degrees(-1.0*ptest.d1_centers_grid[col2,:,chind]),
                f*np.ones(len(ptest.d1_centers_grid[col1,:,chind])),
                c=10*np.log10(abs(ptest.V_LC_operation[col2,:,0,chind])),vmin=-40,vmax=0,cmap=ccm)
        axs[0,a].set_title('$\Theta = $'+str(np.round(np.degrees(arad)))+'$^{\circ}$',fontsize=fsz)
        
        
copoldir = 'E'
pols = E_pols
chind = pols[0]

for find in good_freqs:
    picklefile = polbeamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'_polar.pkl'
    with open(picklefile, 'rb') as inp:
        ptest = pickle.load(inp)
    f = freqs[find]
    for a,arad in enumerate(whichang):
        col1,col2 = get_polar_slice(ptest.d0_centers_grid[:,:,chind],
                          ptest.d1_centers_grid[:,:,chind],arad)
        axs[1,a].scatter(np.degrees(ptest.d1_centers_grid[col1,:,chind]),
                f*np.ones(len(ptest.d1_centers_grid[col1,:,chind])),
                c=10*np.log10(abs(ptest.V_LC_operation[col1,:,0,chind])),vmin=-40,vmax=0,cmap=ccm)
        im = axs[1,a].scatter(np.degrees(-1.0*ptest.d1_centers_grid[col2,:,chind]),
                f*np.ones(len(ptest.d1_centers_grid[col1,:,chind])),
                c=10*np.log10(abs(ptest.V_LC_operation[col2,:,0,chind])),vmin=-40,vmax=0,cmap=ccm)
        axs[1,a].set_xlabel('$\phi$ [Degrees]',fontsize=fsz)
        
axs[0,0].set_ylabel('Frequency [MHz]',fontsize=fsz)
axs[1,0].set_ylabel('Frequency [MHz]',fontsize=fsz)
axs[0,0].text(-33,577,'Input 0',color='white',fontsize=20,fontweight='bold')
axs[1,0].text(-33,577,'Input 1',color='white',fontsize=20,fontweight='bold')

cbar=fig.colorbar(im,ax=axs,aspect=50)
cbar.set_label('Amplitude [dB]',fontsize=fsz)
plt.show()
 

####################################################################################
############################# SECTION 5 ############################################
############################# SECTION 5 ############################################
############################# SECTION 5 ############################################
############################# SECTION 5 ############################################
####################################################################################


# Note, Annie's plots are in her notebook AP_drone_paper_rfi, that won't be reproduced here
# That contains everything ever to get from measurements to EIRP for those calibrated
# measurements.

In [ ]:
######################################################################
########## Get Annie's Chamber Measurements ##########
######################################################################


def VNAtranslator(allscans, rows_to_cut=51):
    freqs = []
    mags = []
    for i in range(len(allscans)):
        data = []
        with open(allscans[i], 'rt') as f:
            reader = csv.reader(f)
            for row in reader:
                data.append(row)
        data_np = np.array(data)
        datafinal = data_np[rows_to_cut:-1]
        df=[]
        freq1 = []
        mag1 = []
        for i in datafinal:
            freq1.append(float(i[0]))
            mag1.append(float(i[1]))   
        freqs.append(freq1)
        mags.append(mag1)
    return(freqs,mags)

def avg_spectra(spec_list):
    avg = np.zeros(len(spec_list[0]))
    for i in range(len(spec_list[0])):
        for j in range(len(spec_list)):
            avg[i] += (10.**(spec_list[j][i]/10.))
        avg[i] = avg[i]/len(spec_list)
        avg[i] = 10.*np.log10(avg[i])        
    return np.array(avg)

# Annie's data loading and manipulating into correct units

# Load in all the RFI chamber data and put it in a nice dictionary

fileset='/hirax/GBO_Oct_2021/RFIChamber/*.csv'

allscans=np.sort(glob.glob(fileset))
vnafreqs,mags=VNAtranslator(allscans = allscans, rows_to_cut=1)

for i in range(len(vnafreqs)):
    vnafreqs[i] = np.array(vnafreqs[i])#/1000000000.
    mags[i] = np.array(mags[i])
    #print(str(i)+": "+allscans[i])

# build the EMI chamber dictionary
chamber = {
    "low":{
        "bgnd":None,
        "horiz":{},
        "vert":{},
        "yaw":{},
        "shield":None,
        "laptop":{},
        "motor":{},
        "nsonly":{},
        "nsdrone":{},
        "long":[],
        "freqs":None
    },
    "high":{
        "bgnd":None,
        "horiz":{},
        "vert":{},
        "yaw":{},
        "shield":None,
        "laptop":None,
        "motor":None,
        "freqs":None
    }
}

used = []
# start by making the low-band version
for i, fname in enumerate(allscans):
    if "20M1G.csv" in fname:
#         print(fname)
        if "QR" in fname:
            chamber["low"]["freqs"] = vnafreqs[i]; used.append(i)
            chamber["low"]["bgnd"] = mags[i]
        if "DroneOnHoriz" in fname:
            angle = int(fname.split('/')[-1][12])
            if angle == 0: chamber["low"]["horiz"]["0"] = mags[i]; used.append(i)
            if angle == 1: chamber["low"]["horiz"]["180"] = mags[i]; used.append(i)
            if angle == 2: chamber["low"]["horiz"]["270"] = mags[i]; used.append(i)
            if angle == 9: chamber["low"]["horiz"]["90"] = mags[i]; used.append(i)
        if "Yaw" in fname and not ("ShieldedPS" in fname):
            angle = int(fname.split('/')[-1][10])
            if angle == 0: chamber["low"]["yaw"]["0"] = mags[i]; used.append(i)
            if angle == 6: chamber["low"]["yaw"]["60"] = mags[i]; used.append(i)
            if angle == 1: chamber["low"]["yaw"]["120"] = mags[i]; used.append(i)
        if "DroneOffNoiseOn" in fname:
            if "Horiz" in fname and "CoPol" in fname: chamber["low"]["nsonly"]["coh"] = mags[i]; used.append(i)
            if "Horiz" in fname and "XPol" in fname: chamber["low"]["nsonly"]["xh"] = mags[i]; used.append(i)
            if "Vert" in fname and "CoPol" in fname: chamber["low"]["nsonly"]["cov"] = mags[i]; used.append(i)
            if "Vert" in fname and "XPol" in fname: chamber["low"]["nsonly"]["xv"] = mags[i]; used.append(i)
        if "DroneOnNoiseOn" in fname:
            if "Horiz" in fname and "CoPol" in fname: chamber["low"]["nsdrone"]["coh"] = mags[i]; used.append(i)
            if "Horiz" in fname and "XPol" in fname: chamber["low"]["nsdrone"]["xh"] = mags[i]; used.append(i)
            if "Vert" in fname and "CoPol" in fname: chamber["low"]["nsdrone"]["cov"] = mags[i]; used.append(i)
            if "Vert" in fname and "XPol" in fname: chamber["low"]["nsdrone"]["xv"] = mags[i]; used.append(i)
        if "LaptopOn" in fname:
            if "RotorsOn" in fname and "HPol" in fname: chamber["low"]["motor"]["hpol"] = mags[i]; used.append(i)
            if "RotorsOn" in fname and "EPol" in fname: chamber["low"]["motor"]["epol"] = mags[i]; used.append(i)
            if not ("RotorsOn" in fname) and "HPol" in fname: chamber["low"]["laptop"]["hpol"] = mags[i]; used.append(i)
            if not ("RotorsOn" in fname) and "EPol" in fname: chamber["low"]["laptop"]["epol"] = mags[i]; used.append(i)
        if "DroneOnVert" in fname and not ("ShieldedPS" in fname):
            angle = fname.split('/')[-1][11]
            if angle != '-':
                if angle == '0': chamber["low"]["vert"]["0"] = mags[i]; used.append(i)
                if angle == '1': chamber["low"]["vert"]["15"] = mags[i]; used.append(i)
                if angle == '3': chamber["low"]["vert"]["30"] = mags[i]; used.append(i)
                if angle == '4': chamber["low"]["vert"]["45"] = mags[i]; used.append(i)
                if angle == '6': chamber["low"]["vert"]["60"] = mags[i]; used.append(i)
                if angle == '9': chamber["low"]["vert"]["90"] = mags[i]; used.append(i)
            else:
                angle = fname.split('/')[-1][12]
                if angle == '3': chamber["low"]["vert"]["-30"] = mags[i]; used.append(i)
                if angle == '4': chamber["low"]["vert"]["-45"] = mags[i]; used.append(i)
                if angle == '6': chamber["low"]["vert"]["-60"] = mags[i]; used.append(i)
                if angle == '9': chamber["low"]["vert"]["-90"] = mags[i]; used.append(i)
                if angle == '1' and fname.split('/')[-1][13] == '5': chamber["low"]["vert"]["-15"] = mags[i]; used.append(i)
                if angle == '1' and fname.split('/')[-1][13] == '8': chamber["low"]["vert"]["180"] = mags[i]; used.append(i)
        if "ShieldedPS" in fname:
            chamber["low"]["shield"] = mags[i]; used.append(i)

low_bgnds = []
            
for i, fname in enumerate(allscans):
    if "EPol20M1G_" in fname:
        chamber["low"]["long"].append(mags[i])
        print(fname)
    if "QR20M1G" in fname:
        low_bgnds.append(mags[i])
        print(fname)

# then join up the high-band spectra
# find all the unique spectra
hi_unique = []
for i, fname in enumerate(allscans):
    if "1G2G.csv" in fname:
        hi_unique.append(fname)

# join each spectrum into one big spectrum
joined_freqs = []
joined_mags = []
joined_basenames = []
for fname in hi_unique:
    basename = fname.split('/')[-1].split('G')[0][:-1]
    joined_basenames.append(basename)
    
    freqs12=[]; freqs24=[]; freqs46=[]; freqs68=[]; freqs810=[]; freqs1012=[]
    mags12=[]; mags24=[]; mags46=[]; mags68=[]; mags810=[]; mags1012=[]
    for i, fname in enumerate(allscans):
        if basename in fname:
            if "1G2G" in fname: mags12 = mags[i]; freqs12 = vnafreqs[i]
            if "2G4G" in fname: mags24 = mags[i]; freqs24 = vnafreqs[i]
            if "4G6G" in fname: mags46 = mags[i]; freqs46 = vnafreqs[i]
            if "6G8G" in fname: mags68 = mags[i]; freqs68 = vnafreqs[i]
            if "8G10G" in fname: mags810 = mags[i]; freqs810 = vnafreqs[i]
            if "10G12G" in fname: mags1012 = mags[i]; freqs1012 = vnafreqs[i]
                
    joined_freqs.append(np.concatenate((freqs12, freqs24, freqs46, freqs68, freqs810, freqs1012)))
    joined_mags.append(np.concatenate((mags12, mags24, mags46, mags68, mags810, mags1012)))

# then assign each spectrum into the dictionary
for i, basename in enumerate(joined_basenames):
    if "QR" in basename: chamber["high"]["bgnd"] = joined_mags[i]
    if "Horiz0" in basename: chamber["high"]["horiz"]["0"] = joined_mags[i]
    if "Horiz90" in basename: chamber["high"]["horiz"]["90"] = joined_mags[i]
    if "LaptopOnHoriz" in basename: chamber["high"]["laptop"] = joined_mags[i]
    if "LaptopOnRotors" in basename: chamber["high"]["motor"] = joined_mags[i]
    if "Vert-0deg" in basename: chamber["high"]["vert"]["0"] = joined_mags[i]
    if "Vert-180deg" in basename: chamber["high"]["vert"]["180"] = joined_mags[i]
    if "Vert-90deg" in basename: chamber["high"]["vert"]["-90"] = joined_mags[i]
    if "Vert90deg" in basename: chamber["high"]["vert"]["90"] = joined_mags[i]
    if "Yaw0deg" in basename and not ("Shielded" in basename): chamber["high"]["yaw"]["0"] = joined_mags[i]
    if "Yaw60deg" in basename: chamber["high"]["yaw"]["60"] = joined_mags[i]
    if "Yaw120deg" in basename: chamber["high"]["yaw"]["120"] = joined_mags[i]
    if "ShieldedPS" in basename: chamber["high"]["shield"] = joined_mags[i]

    
chamber["high"]["freqs"] = joined_freqs[0] # only need one freqs array

# define all the correction curves 

##################################
# carla's emi chamber equipment: #
##################################

# log periodic dipole array (low-band antenna)
lpda_freqs = [20,200,225,250,275,300,325,350,375,400,425,450,475,500,525,550,575,600,625,650,675,700,725,750,775,800,825,850,875,900,925,950,975,1000]
lpda_mags = [-11,3.73,5.79,5.2,5.31,4.09,5.34,5.41,5.86,6.29,6.25,5.85,5.74,5.76,5.62,5.97,6.17,6.2,6.03,6.2,5.71,5.87,6.22,6.42,6.51,6.2,5.83,6.16,5.84,6.06,6.03,5.9,5.51,5.27]

# low-band amp
amp1306e_freqs = [20,130,134.8,139.6,144.4,149.2,154,158.8,163.6,168.4,173.2,178,182.8,187.6,192.4,197.2,202,206.8,211.6,216.4,221.2,226,230.8,235.6,240.4,245.2,250,254.8,259.6,264.4,269.2,274,278.8,283.6,288.4,293.2,298,302.8,307.6,312.4,317.2,322,326.8,331.6,336.4,341.2,346,350.8,355.6,360.4,365.2,370,374.8,379.6,384.4,389.2,394,398.8,403.6,408.4,413.2,418,422.8,427.6,432.4,437.2,442,446.8,451.6,456.4,461.2,466,470.8,475.6,480.4,485.2,490,494.8,499.6,504.4,509.2,514,518.8,523.6,528.4,533.2,538,542.8,547.6,552.4,557.2,562,566.8,571.6,576.4,581.2,586,590.8,595.6,600.4,605.2,610,614.8,619.6,624.4,629.2,634,638.8,643.6,648.4,653.2,658,662.8,667.6,672.4,677.2,682,686.8,691.6,696.4,701.2,706,710.8,715.6,720.4,725.2,730,734.8,739.6,744.4,749.2,754,758.8,763.6,768.4,773.2,778,782.8,787.6,792.4,797.2,802,806.8,811.6,816.4,821.2,826,830.8,835.6,840.4,845.2,850,854.8,859.6,864.4,869.2,874,878.8,883.6,888.4,893.2,898,902.8,907.6,912.4,917.2,922,926.8,931.6,936.4,941.2,946,950.8,955.6,960.4,965.2,970,974.8,979.6,984.4,989.2,994,998.8,1003.6,1008.4,1013.2,1018,1022.8,1027.6,1032.4,1037.2,1042,1046.8,1051.6,1056.4,1061.2,1066,1070.8,1075.6,1080.4,1085.2,1090]
amp1306e_mags = [50.77299,51.5022358002,51.5340572965,51.8168429743,51.5411649186,52.1288497492,52.2331556513,51.9210841202,52.6978485457,52.5066985243,52.3330134085,52.4767813651,52.236362315,52.4667654603,51.7364989038,52.1970763591,51.9618637052,52.3121426383,52.2140104293,51.9567961691,52.7923902668,52.1215574021,52.3924574285,52.6488394845,52.3575815266,52.7850996947,51.7058498701,51.8021452423,52.2515216919,51.5767581212,52.1412423383,52.0633205533,52.0847754917,51.9913646037,52.2682158469,52.1853574007,52.1096260378,52.0225949201,51.9805963851,52.3129039943,52.0124973282,52.5510224111,52.7389164134,52.5358228955,52.1895230602,52.256804462,52.3060012635,52.1119855319,52.208330587,52.455131406,52.1899852624,51.5245325729,51.6456222144,51.7910646106,51.5483851712,51.6174495349,51.5560711701,51.5481432884,51.8686813912,51.6957402271,51.7592209264,51.7867107471,51.4029940195,51.7061277771,51.8080073623,51.9627052784,51.6411282821,51.7282972828,51.8727081752,51.7214349445,51.5751585813,51.84076455,52.0227922504,51.8211688168,51.9583772005,51.7345044121,52.0244277412,51.685077462,51.7261644875,51.6850876244,51.4763034769,51.4494924586,51.4696853247,51.438244663,51.1860506426,51.4433172208,51.4534407907,51.1677466045,51.3316951327,51.0598074924,51.2471965373,51.1376320835,51.2006849395,51.1388057409,51.0842374459,50.9247757784,50.8499695942,50.6627996713,50.528011304,50.482259272,50.4615880455,50.6570666309,50.6975833433,50.5715062129,50.660823626,50.5293576468,50.4444206444,50.5371167789,50.3036436438,50.4406106123,50.5072263847,50.1011505697,50.1628682927,50.2118681652,50.2944151119,50.0381615469,50.1760201525,50.1242166586,50.2095511988,50.1849988886,50.0172019305,50.1505326872,50.2097677205,50.2275466519,50.3095054239,50.4529536245,49.6562079898,49.6870793873,49.9307031115,49.9469055479,50.0773097785,49.8258281863,50.0398213294,49.8756091474,50.0765463733,50.0018541906,49.960330342,50.0440662527,50.0077009624,50.1334818044,50.0277006711,50.1619995218,49.9021834022,49.8484908758,49.9576936266,50.122070728,49.8699308329,49.9950544561,49.9716047428,49.9994713299,49.8619840673,50.0596138993,50.1090530763,49.7740136065,49.9880589813,50.0650304256,50.0327341818,50.070176332,49.9245103197,50.0184203481,49.9550227182,49.9781756895,49.9325854023,49.988188586,50.0883969706,49.9605712262,50.0240080566,49.9663123101,49.9680722882,49.7667914013,49.6831180251,49.5922303679,49.6314711703,49.4572746604,49.3377725314,49.4001453185,49.302544801,49.196111499,49.2202315845,49.080594269,49.1074212725,48.9189748535,48.8772353599,48.8042136073,48.8162557794,48.8466387371,48.8831005917,48.7490654104,48.6283379876,48.5667267299,48.4263009209,48.4226180517,48.2901498186,48.1014925088,48.1421858317,47.9370750006,47.9024891362,47.6489315241,47.6653692958,47.4032027924,47.4701643407,47.2577990117]

# high-band amp
ampamf6d_freqs = [924.4,1023.7,1123,1222.3,1321.6,1420.9,1520.2,1619.5,1718.8,1818.1,1917.4,2016.7,2116,2215.3,2314.6,2413.9,2513.2,2612.5,2711.8,2811.1,2910.4,3009.7,3109,3208.3,3307.6,3406.9,3506.2,3605.5,3704.8,3804.1,3903.4,4002.7,4102,4201.3,4300.6,4399.9,4499.2,4598.5,4697.8,4797.1,4896.4,4995.7,5095,5194.3,5293.6,5392.9,5492.2,5591.5,5690.8,5790.1,5889.4,5988.7,6088,6187.3,6286.6,6385.9,6485.2,6584.5,6683.8,6783.1,6882.4,6981.7,7081,7180.3,7279.6,7378.9,7478.2,7577.5,7676.8,7776.1,7875.4,7974.7,8074,8173.3,8272.6,8371.9,8471.2,8570.5,8669.8,8769.1,8868.4,8967.7,9067,9166.3,9265.6,9364.9,9464.2,9563.5,9662.8,9762.1,9861.4,9960.7,10060,10159.3,10258.6,10357.9,10457.2,10556.5,10655.8,10755.1,10854.4,10953.7,11053,11152.3,11251.6,11350.9,11450.2,11549.5,11648.8,11748.1,11847.4,11946.7,12046,12145.3,12244.6,12343.9,12443.2,12542.5,12641.8,12741.1,12840.4,12939.7,13039,13138.3,13237.6,13336.9,13436.2,13535.5,13634.8,13734.1,13833.4,13932.7,14032,14131.3,14230.6,14329.9,14429.2,14528.5,14627.8,14727.1,14826.4,14925.7,15025,15124.3,15223.6,15322.9,15422.2,15521.5,15620.8,15720.1,15819.4,15918.7,16018,16117.3,16216.6,16315.9,16415.2,16514.5,16613.8,16713.1,16812.4,16911.7,17011,17110.3,17209.6,17308.9,17408.2,17507.5,17606.8,17706.1,17805.4,17904.7,18004,18103.3,18202.6,18301.9,18401.2,18500.5,18599.8,18699.1,18798.4,18897.7,18997,19096.3,19195.6,19294.9,19394.2,19493.5,19592.8,19692.1,19791.4,19890.7,19990]
ampamf6d_mags = [45.95778714,47.22194971,47.84961611,47.81222727,47.92151225,47.41683198,47.37733531,47.31254399,47.24192243,47.45712901,47.12128631,47.42098376,47.29376481,47.26045483,47.29753494,47.36204678,47.36439865,47.47405179,47.61609732,47.69734996,47.4333864,47.44013024,47.46615753,47.45952267,47.42564484,47.35353804,47.27491855,47.2922184,47.22692062,47.1907485,47.44704892,47.15173393,47.24867686,47.22744958,47.27775317,47.16403461,47.01011398,47.20914701,46.84812525,46.87619541,46.59166818,46.7171574,46.40908046,46.10322534,46.03834163,45.5517617,45.16025821,45.19422566,44.98866923,44.79249792,44.68534081,44.66887308,44.76126524,44.7182803,44.86046759,44.89899926,45.26529031,45.27526553,45.51633191,45.95311173,45.92180424,46.10037392,46.2936488,46.08204093,46.3068397,46.41106436,46.30079967,46.24011502,46.0636797,45.85960227,45.68706226,45.46345938,45.38615594,45.22259483,45.00291109,44.84042047,44.6443673,44.59344018,44.2096427,44.21967281,43.76412028,43.86343967,43.69385373,43.28286175,43.67135681,43.21200456,43.33161405,42.85114753,43.59516783,43.16890062,43.23112868,43.33212866,43.60498673,43.82511709,43.88740362,44.1422186,44.15700652,44.09558292,44.44608767,44.28736322,44.47334811,44.34158823,44.18443737,44.26418633,44.23378646,44.15973258,44.13633002,44.29019817,44.32399041,44.25363211,44.39988022,44.64888832,44.72439439,44.71788797,44.92647408,44.95963255,44.84996672,44.96947481,44.6602547,44.87758607,44.42620204,44.63412212,44.51962228,44.45943473,44.54216566,44.35512612,44.30818217,44.39636429,44.6332696,44.84807873,44.85478566,44.93441168,45.00847916,45.0701666,45.08025539,44.8715493,44.8123555,44.47932552,44.54995376,44.26452815,44.65795936,44.6019418,44.56246757,44.79949488,44.85031261,44.69826739,45.05329801,45.13932057,45.31219607,45.26635081,45.45104125,45.51331423,45.76876006,45.82640692,45.61921954,45.45845315,45.32235599,45.37616308,45.17001928,45.05595048,44.87284855,44.77902735,44.66473177,44.74645936,44.65066294,44.6295042,44.7953945,44.31612763,44.78467826,44.71416037,44.76809719,44.8925023,44.89118117,44.89650393,44.63519906,44.83006836,44.37748305,44.19768938,44.44412256,44.09226919,44.11228102,43.88299162,43.7902218,44.17253079,44.36139631,44.0591114,44.30379373,44.37238908,44.18971542,44.10020769,44.23571717,44.18594662,44.24993839]

# cable losses
cablech6_freqs = [20,164.8,189.6,214.4,239.2,264,288.8,313.6,338.4,363.2,388,412.8,437.6,462.4,487.2,512,536.8,561.6,586.4,611.2,636,660.8,685.6,710.4,735.2,760,784.8,809.6,834.4,859.2,884,908.8,933.6,958.4,983.2,1008,1032.8,1057.6,1082.4,1107.2,1132,1156.8,1181.6,1206.4,1231.2,1256,1280.8,1305.6,1330.4,1355.2,1380,1404.8,1429.6,1454.4,1479.2,1504,1528.8,1553.6,1578.4,1603.2,1628,1652.8,1677.6,1702.4,1727.2,1752,1776.8,1801.6,1826.4,1851.2,1876,1900.8,1925.6,1950.4,1975.2,2000,2024.8,2049.6,2074.4,2099.2,2124,2148.8,2173.6,2198.4,2223.2,2248,2272.8,2297.6,2322.4,2347.2,2372,2396.8,2421.6,2446.4,2471.2,2496,2520.8,2545.6,2570.4,2595.2,2620,2644.8,2669.6,2694.4,2719.2,2744,2768.8,2793.6,2818.4,2843.2,2868,2892.8,2917.6,2942.4,2967.2,2992,3016.8,3041.6,3066.4,3091.2,3116,3140.8,3165.6,3190.4,3215.2,3240,3264.8,3289.6,3314.4,3339.2,3364,3388.8,3413.6,3438.4,3463.2,3488,3512.8,3537.6,3562.4,3587.2,3612,3636.8,3661.6,3686.4,3711.2,3736,3760.8,3785.6,3810.4,3835.2,3860,3884.8,3909.6,3934.4,3959.2,3984,4008.8,4033.6,4058.4,4083.2,4108,4132.8,4157.6,4182.4,4207.2,4232,4256.8,4281.6,4306.4,4331.2,4356,4380.8,4405.6,4430.4,4455.2,4480,4504.8,4529.6,4554.4,4579.2,4604,4628.8,4653.6,4678.4,4703.2,4728,4752.8,4777.6,4802.4,4827.2,4852,4876.8,4901.6,4926.4,4951.2,4976,5000.8,5025.6,5050.4,5075.2,5100,5124.8,5149.6,5174.4,5199.2,5224,5248.8,5273.6,5298.4,5323.2,5348,5372.8,5397.6,5422.4,5447.2,5472,5496.8,5521.6,5546.4,5571.2,5596,5620.8,5645.6,5670.4,5695.2,5720,5744.8,5769.6,5794.4,5819.2,5844,5868.8,5893.6,5918.4,5943.2,5968,5992.8,6017.6,6042.4,6067.2,6092,6116.8,6141.6,6166.4,6191.2,6216,6240.8,6265.6,6290.4,6315.2,6340,6364.8,6389.6,6414.4,6439.2,6464,6488.8,6513.6,6538.4,6563.2,6588,6612.8,6637.6,6662.4,6687.2,6712,6736.8,6761.6,6786.4,6811.2,6836,6860.8,6885.6,6910.4,6935.2,6960,6984.8,7009.6,7034.4,7059.2,7084,7108.8,7133.6,7158.4,7183.2,7208,7232.8,7257.6,7282.4,7307.2,7332,7356.8,7381.6,7406.4,7431.2,7456,7480.8,7505.6,7530.4,7555.2,7580,7604.8,7629.6,7654.4,7679.2,7704,7728.8,7753.6,7778.4,7803.2,7828,7852.8,7877.6,7902.4,7927.2,7952,7976.8,8001.6,8026.4,8051.2,8076,8100.8,8125.6,8150.4,8175.2,8200,8224.8,8249.6,8274.4,8299.2,8324,8348.8,8373.6,8398.4,8423.2,8448,8472.8,8497.6,8522.4,8547.2,8572,8596.8,8621.6,8646.4,8671.2,8696,8720.8,8745.6,8770.4,8795.2,8820,8844.8,8869.6,8894.4,8919.2,8944,8968.8,8993.6,9018.4,9043.2,9068,9092.8,9117.6,9142.4,9167.2,9192,9216.8,9241.6,9266.4,9291.2,9316,9340.8,9365.6,9390.4,9415.2,9440,9464.8,9489.6,9514.4,9539.2,9564,9588.8,9613.6,9638.4,9663.2,9688,9712.8,9737.6,9762.4,9787.2,9812,9836.8,9861.6,9886.4,9911.2,9936,9960.8,9985.6,10010.4,10035.2,10060,10084.8,10109.6,10134.4,10159.2,10184,10208.8,10233.6,10258.4,10283.2,10308,10332.8,10357.6,10382.4,10407.2,10432,10456.8,10481.6,10506.4,10531.2,10556,10580.8,10605.6,10630.4,10655.2,10680,10704.8,10729.6,10754.4,10779.2,10804,10828.8,10853.6,10878.4,10903.2,10928,10952.8,10977.6,11002.4,11027.2,11052,11076.8,11101.6,11126.4,11151.2,11176,11200.8,11225.6,11250.4,11275.2,11300,11324.8,11349.6,11374.4,11399.2,11424,11448.8,11473.6,11498.4,11523.2,11548,11572.8,11597.6,11622.4,11647.2,11672,11696.8,11721.6,11746.4,11771.2,11796,11820.8,11845.6,11870.4,11895.2,11920,11944.8,11969.6,11994.4,12019.2,12044,12068.8,12093.6,12118.4,12143.2,12168,12192.8,12217.6,12242.4,12267.2,12292,12316.8,12341.6,12366.4,12391.2,12416,12440.8,12465.6,12490.4,12515.2,12540,12564.8,12589.6,12614.4,12639.2,12664,12688.8,12713.6,12738.4,12763.2,12788,12812.8,12837.6,12862.4,12887.2,12912,12936.8,12961.6,12986.4,13011.2,13036,13060.8,13085.6,13110.4,13135.2,13160,13184.8,13209.6,13234.4,13259.2,13284,13308.8,13333.6,13358.4,13383.2,13408,13432.8,13457.6,13482.4,13507.2,13532,13556.8,13581.6,13606.4,13631.2,13656,13680.8,13705.6,13730.4,13755.2,13780,13804.8,13829.6,13854.4,13879.2,13904,13928.8,13953.6,13978.4,14003.2,14028,14052.8,14077.6,14102.4,14127.2,14152,14176.8,14201.6,14226.4,14251.2,14276,14300.8,14325.6,14350.4,14375.2,14400,14424.8,14449.6,14474.4,14499.2,14524,14548.8,14573.6,14598.4,14623.2,14648,14672.8,14697.6,14722.4,14747.2,14772,14796.8,14821.6,14846.4,14871.2,14896,14920.8,14945.6,14970.4,14995.2,15020,15044.8,15069.6,15094.4,15119.2,15144,15168.8,15193.6,15218.4,15243.2,15268,15292.8,15317.6,15342.4,15367.2,15392,15416.8,15441.6,15466.4,15491.2,15516,15540.8,15565.6,15590.4,15615.2,15640,15664.8,15689.6,15714.4,15739.2,15764,15788.8,15813.6,15838.4,15863.2,15888,15912.8,15937.6,15962.4,15987.2,16012,16036.8,16061.6,16086.4,16111.2,16136,16160.8,16185.6,16210.4,16235.2,16260,16284.8,16309.6,16334.4,16359.2,16384,16408.8,16433.6,16458.4,16483.2,16508,16532.8,16557.6,16582.4,16607.2,16632,16656.8,16681.6,16706.4,16731.2,16756,16780.8,16805.6,16830.4,16855.2,16880,16904.8,16929.6,16954.4,16979.2,17004,17028.8,17053.6,17078.4,17103.2,17128,17152.8,17177.6,17202.4,17227.2,17252,17276.8,17301.6,17326.4,17351.2,17376,17400.8,17425.6,17450.4,17475.2,17500,17524.8,17549.6,17574.4,17599.2,17624,17648.8,17673.6,17698.4,17723.2,17748,17772.8,17797.6,17822.4,17847.2,17872,17896.8,17921.6,17946.4,17971.2,17996,18020.8,18045.6,18070.4,18095.2,18120,18144.8,18169.6,18194.4,18219.2,18244,18268.8,18293.6,18318.4,18343.2,18368,18392.8,18417.6,18442.4,18467.2,18492,18516.8,18541.6,18566.4,18591.2,18616,18640.8,18665.6,18690.4,18715.2,18740,18764.8,18789.6,18814.4,18839.2,18864,18888.8,18913.6,18938.4,18963.2,18988,19012.8,19037.6,19062.4,19087.2,19112,19136.8,19161.6,19186.4,19211.2,19236,19260.8,19285.6,19310.4,19335.2,19360,19384.8,19409.6,19434.4,19459.2,19484,19508.8,19533.6,19558.4,19583.2,19608,19632.8,19657.6,19682.4,19707.2,19732,19756.8,19781.6,19806.4,19831.2,19856,19880.8,19905.6,19930.4,19955.2,19980]
cablech6_mags = [0,-0.020415,-0.116372,-0.104923,-0.118715,-0.04337,-0.119868,-0.094994,-0.089767,-0.146837,-0.059809,-0.122356,-0.226849,-0.176591,-0.206335,-0.174592,-0.132276,-0.203308,-0.25658,-0.198695,-0.18013,-0.187925,-0.2586,-0.164849,-0.262963,-0.262222,-0.416763,-0.269963,-0.268014,-0.272342,-0.252612,-0.286905,-0.273519,-0.249407,-0.278059,-0.304683,-0.325073,-0.362833,-0.342147,-0.283697,-0.278026,-0.342403,-0.25999,-0.294714,-0.311474,-0.269784,-0.381589,-0.266805,-0.307006,-0.375622,-0.289624,-0.284262,-0.333004,-0.409639,-0.318613,-0.317457,-0.328351,-0.384389,-0.284151,-0.401202,-0.366552,-0.294518,-0.360034,-0.374648,-0.31597,-0.327578,-0.435576,-0.379535,-0.359344,-0.407602,-0.392862,-0.40935,-0.410917,-0.355468,-0.358773,-0.402523,-0.370778,-0.396463,-0.419693,-0.37013,-0.361664,-0.443377,-0.386099,-0.38946,-0.387569,-0.453982,-0.364419,-0.424306,-0.450086,-0.380501,-0.394945,-0.365852,-0.43197,-0.408407,-0.477305,-0.471112,-0.454955,-0.426139,-0.476122,-0.446477,-0.44392,-0.42703,-0.439244,-0.47535,-0.431599,-0.419851,-0.431556,-0.482468,-0.506807,-0.446511,-0.432444,-0.52501,-0.548263,-0.463121,-0.47595,-0.565813,-0.484675,-0.519454,-0.520506,-0.534091,-0.486698,-0.490898,-0.573605,-0.492631,-0.469022,-0.500446,-0.480992,-0.522034,-0.519515,-0.493653,-0.595318,-0.576497,-0.455724,-0.558905,-0.52107,-0.600039,-0.577964,-0.594174,-0.526053,-0.441731,-0.642067,-0.443939,-0.566202,-0.543638,-0.560142,-0.553903,-0.54399,-0.587672,-0.541393,-0.546179,-0.58275,-0.539696,-0.598114,-0.530428,-0.61983,-0.58027,-0.46804,-0.575062,-0.609613,-0.519802,-0.565336,-0.525571,-0.543622,-0.594907,-0.586313,-0.552006,-0.591029,-0.626294,-0.607905,-0.539448,-0.619231,-0.455184,-0.534685,-0.527118,-0.601068,-0.564523,-0.655992,-0.547517,-0.56026,-0.581198,-0.607209,-0.568433,-0.614264,-0.645509,-0.586329,-0.606149,-0.608466,-0.544464,-0.6866,-0.562113,-0.644492,-0.531288,-0.653436,-0.563956,-0.619145,-0.629709,-0.613526,-0.570785,-0.67888,-0.646205,-0.612094,-0.676094,-0.546271,-0.636365,-0.587686,-0.749892,-0.67174,-0.66622,-0.641263,-0.628136,-0.62697,-0.643512,-0.726239,-0.690969,-0.653663,-0.633196,-0.709147,-0.668661,-0.616804,-0.759677,-0.636785,-0.662904,-0.642974,-0.689194,-0.662556,-0.67475,-0.689646,-0.673493,-0.659893,-0.65555,-0.660841,-0.675919,-0.640155,-0.679721,-0.603202,-0.811365,-0.693811,-0.685776,-0.701056,-0.774035,-0.7048,-0.712433,-0.712513,-0.583901,-0.79803,-0.603578,-0.726293,-0.705523,-0.693209,-0.753238,-0.687282,-0.730183,-0.63633,-0.69152,-0.735011,-0.708925,-0.698884,-0.725897,-0.691059,-0.705912,-0.636692,-0.713291,-0.721281,-0.792462,-0.734116,-0.631705,-0.67266,-0.730466,-0.683198,-0.663434,-0.794086,-0.694743,-0.694355,-0.792881,-0.641949,-0.77662,-0.751445,-0.748439,-0.783365,-0.736871,-0.766553,-0.693096,-0.73981,-0.800142,-0.644389,-0.712977,-0.745846,-0.762887,-0.740219,-0.721056,-0.736454,-0.741341,-0.802369,-0.69564,-0.776219,-0.747513,-0.783835,-0.901215,-0.698176,-0.7258,-0.796439,-0.754949,-0.85467,-0.663963,-0.830065,-0.75396,-0.736693,-0.816452,-0.72841,-0.759993,-0.789143,-0.733034,-0.80848,-0.813913,-0.769228,-0.771506,-0.780755,-0.787983,-0.711063,-0.757456,-0.780661,-0.739773,-0.806333,-0.814676,-0.743421,-0.778925,-0.796655,-0.773187,-0.78123,-0.787607,-0.839366,-0.762989,-0.777116,-0.827016,-0.767952,-0.911171,-0.725542,-0.872135,-0.849805,-0.798298,-0.735154,-0.782939,-0.90554,-0.83463,-0.854681,-0.830188,-0.772379,-0.781445,-0.82293,-0.889109,-0.850488,-0.801331,-0.803562,-0.785244,-0.803489,-0.757385,-0.63493,-1.152685,-0.700412,-0.846606,-0.789713,-0.8132,-0.730777,-0.925905,-0.957835,-0.847617,-0.852585,-0.781831,-0.734527,-0.894975,-0.827797,-0.835426,-0.835054,-0.948094,-0.782675,-0.7542,-0.892699,-0.854749,-0.904954,-0.937588,-0.893647,-0.859134,-0.780312,-0.810692,-0.820607,-0.857632,-0.923095,-0.875385,-0.826393,-0.847218,-0.840271,-0.811153,-0.848269,-1.003161,-0.881234,-0.867763,-0.879973,-0.893519,-0.907135,-0.870399,-0.870273,-0.980718,-0.888251,-0.908469,-0.950156,-0.842498,-0.882226,-0.953788,-0.903341,-0.994218,-0.951056,-0.927317,-0.906211,-0.878424,-0.987914,-0.868548,-1.10756,-0.829632,-0.86717,-1.004785,-1.007646,-0.90024,-1.011331,-0.767029,-0.943994,-1.058264,-1.033182,-0.916456,-1.010658,-0.773141,-1.109767,-0.905872,-1.048698,-0.719283,-1.025863,-0.889549,-1.132278,-0.991456,-0.91068,-0.900804,-0.866971,-1.053401,-1.078111,-0.894187,-0.851085,-0.919481,-0.923647,-0.937906,-0.855988,-0.982525,-0.901472,-0.919871,-1.108786,-0.967665,-0.950919,-0.986662,-0.717984,-1.27931,-1.076822,-0.961935,-0.895323,-1.058756,-1.003367,-1.049846,-0.922016,-1.006136,-1.009646,-1.103488,-0.967767,-0.924341,-0.982892,-0.973351,-1.17365,-1.113414,-1.109433,-1.006288,-0.945202,-0.881734,-1.045618,-1.21842,-0.98634,-1.097997,-1.222054,-0.916085,-0.858727,-0.999645,-0.949456,-1.365836,-1.049996,-0.89901,-1.005241,-0.97141,-0.82779,-0.93247,-1.185523,-1.307621,-0.739826,-0.935046,-0.864913,-1.028759,-1.020947,-1.031388,-0.983967,-1.262844,-0.701195,-0.851096,-0.933417,-0.973403,-1.034946,-0.994423,-1.069197,-0.938912,-0.674399,-0.977214,-1.048341,-1.220237,-0.815311,-0.944834,-1.013799,-0.968113,-0.869515,-0.92553,-0.946823,-1.10176,-1.056044,-0.787986,-1.027096,-1.017945,-0.975484,-1.036316,-0.965188,-0.993932,-1.012021,-0.894983,-0.96367,-0.993825,-0.94766,-0.996527,-1.043553,-1.033077,-1.00679,-0.954775,-1.072416,-1.086477,-1.105614,-1.011589,-1.01144,-0.998157,-1.02613,-0.97864,-1.076885,-1.085651,-1.023255,-0.986321,-1.108482,-0.960883,-1.014567,-1.094488,-1.111887,-1.028884,-1.096578,-0.987821,-1.048429,-1.050723,-1.023808,-1.047724,-1.05434,-0.936694,-1.083662,-1.029301,-1.033294,-0.893607,-1.015375,-1.122951,-1.109728,-0.948023,-1.08003,-1.098183,-1.002226,-1.07689,-1.138612,-1.082193,-1.032135,-1.074362,-1.114265,-1.144305,-1.005581,-1.100826,-1.127288,-1.121301,-1.069359,-1.018199,-1.040812,-1.045329,-1.049671,-1.229826,-1.169745,-1.066442,-1.083212,-1.094698,-1.039781,-1.098754,-1.214502,-1.20404,-1.004619,-1.032207,-1.14531,-1.184387,-1.009371,-1.097988,-1.172097,-1.161891,-1.147896,-0.927085,-1.148413,-1.053007,-1.034368,-1.288364,-1.208228,-1.168139,-0.913536,-1.209854,-1.188709,-1.269148,-1.199938,-1.032178,-1.126486,-1.109044,-1.125597,-1.245329,-1.242252,-1.087329,-1.146717,-1.186238,-1.151041,-1.020825,-1.176294,-1.116011,-1.178937,-1.27202,-1.232979,-1.021955,-1.187848,-1.065496,-1.232939,-1.20632,-1.067785,-1.077517,-1.165736,-1.155983,-1.082762,-1.143231,-1.195869,-0.987037,-1.203324,-1.142155,-1.122002,-1.037954,-1.163854,-1.107683,-1.188191,-1.117561,-1.027987,-1.267075,-1.097764,-1.102054,-1.224031,-1.181807,-1.20764,-1.106037,-1.319787,-1.089974,-1.107508,-1.033333,-1.233751,-1.107584,-1.147986,-1.225514,-1.040754,-1.269198,-1.149096,-1.027249,-1.149253,-1.108913,-1.119157,-1.205432,-1.192542,-1.149194,-1.143716,-1.110802,-1.047597,-1.168629,-1.168197,-1.088382,-1.099469,-1.234897,-1.047954,-1.0006,-1.072314,-1.208558,-1.150716,-1.248578,-1.035766,-0.978594,-1.082756,-1.181818,-1.192373,-1.143615,-1.013383,-0.914815,-1.083617,-1.061227,-1.105646,-1.016384,-1.157594,-1.128523,-1.179205,-1.052266,-0.825379,-1.312292,-1.132685,-1.124103,-1.05446,-1.124416,-1.01579,-1.175453,-1.164673,-1.055293,-1.148347,-1.181176,-1.105106,-1.108582,-1.098601,-0.891879,-1.352514,-1.091146,-1.301805,-1.13683,-1.058713,-1.121684,-1.162267,-1.220394,-1.152263,-1.043806,-1.177432,-1.322354,-1.141759,-1.090397,-1.014632,-1.312252,-1.217742,-1.121646,-1.16469,-1.124111,-1.154408,-1.120631,-1.276607,-1.331616,-0.964096,-1.072643,-1.482297,-1.15151,-1.247609,-1.196013,-1.168075,-1.186388,-1.292605,-1.52701,-1.07329,-1.279569,-1.067119,-1.10843,-1.771709,-1.001574,-1.216918,-1.162825,-1.475201,-1.257085,-1.298681,-1.263584,-1.203539,-1.404467,-1.298597,-1.292691,-1.258498,-1.169976,-1.318019,-1.303333,-1.571074,-1.014513,-1.314864,-1.120775,-1.408083,-1.571175,-1.230176,-1.253138,-1.241572,-1.435188,-1.366575,-1.367084,-1.29235]

cableab_freqs = [20,130,229.3,328.6,427.9,527.2,626.5,725.8,825.1,924.4,1023.7,1123,1222.3,1321.6,1420.9,1520.2,1619.5,1718.8,1818.1,1917.4,2016.7,2116,2215.3,2314.6,2413.9,2513.2,2612.5,2711.8,2811.1,2910.4,3009.7,3109,3208.3,3307.6,3406.9,3506.2,3605.5,3704.8,3804.1,3903.4,4002.7,4102,4201.3,4300.6,4399.9,4499.2,4598.5,4697.8,4797.1,4896.4,4995.7,5095,5194.3,5293.6,5392.9,5492.2,5591.5,5690.8,5790.1,5889.4,5988.7,6088,6187.3,6286.6,6385.9,6485.2,6584.5,6683.8,6783.1,6882.4,6981.7,7081,7180.3,7279.6,7378.9,7478.2,7577.5,7676.8,7776.1,7875.4,7974.7,8074,8173.3,8272.6,8371.9,8471.2,8570.5,8669.8,8769.1,8868.4,8967.7,9067,9166.3,9265.6,9364.9,9464.2,9563.5,9662.8,9762.1,9861.4,9960.7,10060,10159.3,10258.6,10357.9,10457.2,10556.5,10655.8,10755.1,10854.4,10953.7,11053,11152.3,11251.6,11350.9,11450.2,11549.5,11648.8,11748.1,11847.4,11946.7,12046,12145.3,12244.6,12343.9,12443.2,12542.5,12641.8,12741.1,12840.4,12939.7,13039,13138.3,13237.6,13336.9,13436.2,13535.5,13634.8,13734.1,13833.4,13932.7,14032,14131.3,14230.6,14329.9,14429.2,14528.5,14627.8,14727.1,14826.4,14925.7,15025,15124.3,15223.6,15322.9,15422.2,15521.5,15620.8,15720.1,15819.4,15918.7,16018,16117.3,16216.6,16315.9,16415.2,16514.5,16613.8,16713.1,16812.4,16911.7,17011,17110.3,17209.6,17308.9,17408.2,17507.5,17606.8,17706.1,17805.4,17904.7,18004,18103.3,18202.6,18301.9,18401.2,18500.5,18599.8,18699.1,18798.4,18897.7,18997,19096.3,19195.6,19294.9,19394.2,19493.5,19592.8,19692.1,19791.4,19890.7,19990]
cableab_mags = [0,-0.394453298613,-0.455248495029,-0.794609628205,-1.18154055241,-1.25467668724,-1.29560493651,-1.31902170074,-2.20806560631,-2.27816581087,-2.27570051882,-2.33857369215,-2.37506950718,-2.43432479679,-2.47947443304,-2.54383941605,-2.5789964841,-2.60127165819,-2.60626293201,-2.76426918055,-2.68898899773,-2.77945983377,-2.82533419924,-2.7949306087,-2.79764861835,-2.84003640884,-2.96997914993,-2.98048234199,-3.06062581439,-3.02351028293,-3.22114581389,-3.16630021565,-3.18480463727,-3.30279089186,-3.28421463906,-3.41777999826,-3.50472130749,-3.51303858949,-3.50216211708,-3.51570341091,-3.50593404564,-3.68984996687,-3.64382674206,-3.81098409191,-3.85167807129,-3.75354494026,-3.86342072665,-3.88325738652,-3.87973134068,-4.02833747196,-4.02175053127,-4.14816566405,-4.20143591585,-4.21787985198,-4.31725477583,-4.40633488919,-4.42881098196,-4.40245020855,-4.43152877405,-4.49283142256,-4.52356174407,-4.46342935289,-4.55288590522,-4.48659954193,-4.75084839039,-4.73297156301,-4.80606383215,-4.90263465458,-4.87066040283,-4.90439545733,-4.89138460241,-5.08058007046,-4.98859755014,-5.04998549001,-4.95963659864,-5.15164407123,-5.18702733492,-5.25163490661,-5.34332469612,-5.36216994411,-5.63287717123,-5.52049528802,-6.07614129438,-5.70002021125,-5.99239236054,-6.05092975909,-6.24972828106,-6.10850892948,-5.98413272923,-6.36988350249,-6.32673259527,-6.52098035983,-6.34775691901,-6.72517507689,-6.37600199057,-6.60579210825,-6.54963890105,-6.64064939141,-6.63662430007,-6.66005819932,-6.55665972624,-6.96420277643,-6.58961325751,-6.59237954876,-6.74164848936,-6.82192410223,-6.55234555399,-6.2743351349,-6.67476943318,-6.58611959099,-6.05059530884,-6.33555796518,-6.66286128164,-6.01971155317,-6.16954735075,-6.72509099405,-6.66309582246,-6.37841525008,-6.73937839648,-6.65341302688,-7.19244052016,-7.03375192815,-7.49592104488,-7.66578354664,-8.34683254304,-8.81208217464,-9.2079264145,-12.2974959313,-16.3569473583,-22.8229857133,-14.1544992361,-10.8336657074,-9.71240997778,-7.96632278763,-7.80899790608,-7.4911969782,-7.78921137796,-7.35132832974,-7.42519609207,-7.41734737569,-7.51227075173,-7.31802641784,-7.62435059742,-8.46316325662,-8.12233035419,-7.69158095179,-7.86839172444,-7.9099697875,-7.71904369971,-8.03055662019,-7.88384318331,-8.08854539565,-8.03163132674,-8.00577000191,-8.15974982562,-8.26205565778,-8.63852903796,-8.3190358326,-8.62406428666,-8.69661297295,-8.62709952842,-8.70789567475,-8.4693594441,-8.79616851939,-8.22652623121,-8.88894387086,-8.35666893959,-8.74574192246,-8.96215243442,-8.82908955529,-8.91757303731,-8.84088427436,-9.47973268353,-9.23105687532,-9.64821732059,-9.04366585805,-9.73245292241,-9.71852317862,-9.18578154519,-9.37979244203,-9.40611041765,-9.96782551976,-9.52326949158,-10.0150474012,-9.70428876557,-11.1944768081,-10.0003090971,-11.3006538563,-11.1203360329,-10.6628417417,-11.1443065908,-10.9024430852,-11.2689611274,-11.5220316193,-10.9641118355,-11.5684578085,-11.9717628219,-12.6433716265,-12.9551235815,-13.1364569334,-13.3314017448,-14.7821996084]

cableac_freqs = [20,130,229.3,328.6,427.9,527.2,626.5,725.8,825.1,924.4,1023.7,1123,1222.3,1321.6,1420.9,1520.2,1619.5,1718.8,1818.1,1917.4,2016.7,2116,2215.3,2314.6,2413.9,2513.2,2612.5,2711.8,2811.1,2910.4,3009.7,3109,3208.3,3307.6,3406.9,3506.2,3605.5,3704.8,3804.1,3903.4,4002.7,4102,4201.3,4300.6,4399.9,4499.2,4598.5,4697.8,4797.1,4896.4,4995.7,5095,5194.3,5293.6,5392.9,5492.2,5591.5,5690.8,5790.1,5889.4,5988.7,6088,6187.3,6286.6,6385.9,6485.2,6584.5,6683.8,6783.1,6882.4,6981.7,7081,7180.3,7279.6,7378.9,7478.2,7577.5,7676.8,7776.1,7875.4,7974.7,8074,8173.3,8272.6,8371.9,8471.2,8570.5,8669.8,8769.1,8868.4,8967.7,9067,9166.3,9265.6,9364.9,9464.2,9563.5,9662.8,9762.1,9861.4,9960.7,10060,10159.3,10258.6,10357.9,10457.2,10556.5,10655.8,10755.1,10854.4,10953.7,11053,11152.3,11251.6,11350.9,11450.2,11549.5,11648.8,11748.1,11847.4,11946.7,12046,12145.3,12244.6,12343.9,12443.2,12542.5,12641.8,12741.1,12840.4,12939.7,13039,13138.3,13237.6,13336.9,13436.2,13535.5,13634.8,13734.1,13833.4,13932.7,14032,14131.3,14230.6,14329.9,14429.2,14528.5,14627.8,14727.1,14826.4,14925.7,15025,15124.3,15223.6,15322.9,15422.2,15521.5,15620.8,15720.1,15819.4,15918.7,16018,16117.3,16216.6,16315.9,16415.2,16514.5,16613.8,16713.1,16812.4,16911.7,17011,17110.3,17209.6,17308.9,17408.2,17507.5,17606.8,17706.1,17805.4,17904.7,18004,18103.3,18202.6,18301.9,18401.2,18500.5,18599.8,18699.1,18798.4,18897.7,18997,19096.3,19195.6,19294.9,19394.2,19493.5,19592.8,19692.1,19791.4,19890.7,19990]
cableac_mags = [0,-0.153880282028,-0.214729135422,-0.332139948251,-0.472257914269,-0.464866105519,-0.417830560276,-0.496738903262,-0.815266451701,-0.848097010672,-0.891511571161,-0.926279643947,-0.96516518152,-0.960014338498,-0.943600570272,-1.07602533131,-1.05797473884,-1.06977181344,-1.06436152731,-1.08831034044,-1.12420237343,-1.11676256669,-1.23164573019,-1.2261942754,-1.225427687,-1.2559597304,-1.29252366757,-1.39118320394,-1.4051903252,-1.33954046873,-1.34289855029,-1.37960469165,-1.47807823278,-1.5452780753,-1.53986095893,-1.51765827069,-1.62519000671,-1.63414609285,-1.73157629877,-1.67978903934,-1.65371521163,-1.76082108165,-1.7659042693,-1.8119174645,-1.95949579604,-1.83238634844,-2.00940330413,-1.98783188636,-2.02901161268,-2.02008781294,-2.00045533113,-2.14480475694,-2.19866313019,-2.24668840807,-2.23558403705,-2.23963514492,-2.36794921477,-2.40490834625,-2.30893075416,-2.47573903974,-2.39777579984,-2.5379332284,-2.50272465749,-2.56062700683,-2.70861849124,-2.63040210432,-2.56468885009,-2.73046487284,-2.5836596167,-2.72662708789,-2.58878787251,-2.88164376385,-2.6099809069,-2.76812647849,-2.65385838675,-2.87146045576,-2.84570405108,-2.98050291188,-3.19646027503,-2.95343497781,-3.33478547062,-2.97318439988,-3.62452585449,-2.9827384674,-3.55763851132,-3.23475140761,-3.71611895089,-3.24498027698,-3.65442859267,-3.54950509643,-3.90601816665,-3.67757741506,-3.72322687375,-3.81293719801,-3.75793328118,-3.62330277332,-3.75295879328,-3.49127379176,-3.65783105033,-3.45109427782,-3.6312682536,-3.61231160086,-3.47533915553,-3.30140045641,-3.62263029651,-3.41130369093,-3.42523866487,-3.12599420425,-3.43932512938,-3.31660024075,-2.94449009247,-3.12044155694,-3.37298656099,-2.83684156682,-3.05158950099,-3.36311183081,-3.47371038993,-3.30008863157,-3.46560610148,-3.29311864381,-3.76743363377,-3.45793509043,-3.65834316501,-3.84418498017,-3.86830403559,-4.15722362328,-3.70533239391,-4.47442480768,-4.04860248913,-4.59214006331,-4.24136782829,-4.72052712467,-4.48286889281,-4.59339782543,-4.39340108329,-4.95282753076,-5.27458924135,-5.34324307116,-5.43611574978,-6.81664640804,-7.18653384078,-10.1234695062,-12.8937158148,-23.8777772953,-19.6527615151,-13.2517821705,-9.59121604621,-8.41435065016,-6.74756602557,-6.49876896031,-5.86278489348,-6.05787438981,-5.51200486849,-5.31295139514,-5.22326053205,-5.2549657419,-5.54176532385,-5.11708773518,-5.28008961911,-5.12578283451,-5.50577169618,-5.05154686486,-5.05874681377,-4.96832014275,-4.94231405447,-4.95786762454,-4.85511009223,-4.81318098845,-5.04111276305,-4.8388738114,-4.84998061922,-4.77795182721,-5.42272670366,-4.77944121604,-5.48529984595,-4.52651503948,-5.76810065305,-4.82350784427,-5.0062344053,-4.62211142858,-5.29975864804,-4.68034074745,-4.56307895244,-4.70844226315,-4.73439242078,-5.12887691807,-4.41148411265,-4.57092213703,-5.12694700226,-4.62852062361,-4.53196228673,-4.48900585182,-4.83235563276,-4.68474134404,-4.78146377547,-4.65417467138,-4.94487823346,-4.87433656331,-5.1889438993,-5.1249887204,-5.35959691328,-6.07711434577]

# total cable loss (assuming all cables are used in all cases)
cableab_interp = np.interp(cablech6_freqs, cableab_freqs, cableab_mags)
cableac_interp = np.interp(cablech6_freqs, cableac_freqs, cableac_mags)
allcable_freqs = cablech6_freqs
allcable_mags = cablech6_mags + cableab_interp + cableac_interp


f = chamber["low"]["freqs"]

####
# get ready to convert the EMI chamber data to dBuV/m at 10m
####

# total cable loss (assuming all cables are used in all cases)
cableab_interp = np.interp(cablech6_freqs, cableab_freqs, cableab_mags)
cableac_interp = np.interp(cablech6_freqs, cableac_freqs, cableac_mags)
allcable_freqs = cablech6_freqs
allcable_mags = cablech6_mags + cableab_interp + cableac_interp

# convert the antenna gain to an antenna factor
lpda_af = 20.*np.log10(lpda_freqs) - lpda_mags - 29.7707
lpda_af = np.interp(f, lpda_freqs, lpda_af)

# build a cable and amplifier gain correction
lo_cable_amp = -1.*np.interp(f, allcable_freqs, allcable_mags) - np.interp(f, amp1306e_freqs, amp1306e_mags)

# make a big combined correction factor
chamber_corr = lo_cable_amp + lpda_af

# correction to convert measured dBm to EIRP
dbm_to_eirp = chamber_corr + 19.13


worst_yaw = np.maximum(chamber['low']['yaw']['0'], np.maximum(chamber['low']['yaw']['60'], chamber['low']['yaw']['120'])) # the worst-case yaw
worst_horiz = np.maximum(np.maximum(chamber['low']['horiz']['0'], chamber['low']['horiz']['90']), np.maximum(chamber['low']['horiz']['180'], chamber['low']['horiz']['270'])) # the worst-case horizontal
worst_vert = np.maximum(np.maximum(np.maximum(chamber['low']['vert']['0'], chamber['low']['vert']['15']), np.maximum(chamber['low']['vert']['30'], chamber['low']['vert']['45'])), 
                        np.maximum(np.maximum(np.maximum(chamber['low']['vert']['60'], chamber['low']['vert']['180']), np.maximum(chamber['low']['vert']['90'], chamber['low']['vert']['-15'])), 
                                   np.maximum(np.maximum(chamber['low']['vert']['-30'], chamber['low']['vert']['-45']), np.maximum(chamber['low']['vert']['-60'], chamber['low']['vert']['-90'])))) # the worst-case vertical (oof)
all_worst = np.maximum(np.maximum(worst_yaw, worst_horiz), worst_vert)

best_yaw  = np.minimum(chamber['low']['yaw']['0'], np.minimum(chamber['low']['yaw']['60'], chamber['low']['yaw']['120'])) # the worst-case yaw
best_horiz  = np.minimum(np.minimum(chamber['low']['horiz']['0'], chamber['low']['horiz']['90']), np.minimum(chamber['low']['horiz']['180'], chamber['low']['horiz']['270'])) # the worst-case horizontal
best_vert  = np.minimum(np.minimum(np.minimum(chamber['low']['vert']['0'], chamber['low']['vert']['15']), np.minimum(chamber['low']['vert']['30'], chamber['low']['vert']['45'])), 
                        np.minimum(np.minimum(np.minimum(chamber['low']['vert']['60'], chamber['low']['vert']['180']), np.minimum(chamber['low']['vert']['90'], chamber['low']['vert']['-15'])), 
                                   np.minimum(np.minimum(chamber['low']['vert']['-30'], chamber['low']['vert']['-45']), np.minimum(chamber['low']['vert']['-60'], chamber['low']['vert']['-90'])))) # the worst-case vertical (oof)
all_best = np.minimum(np.minimum(best_yaw, best_horiz), best_vert)


averaged_spectra = avg_spectra([chamber['low']['yaw']['0'], chamber['low']['yaw']['60'], chamber['low']['yaw']['120'], chamber['low']['horiz']['0'], 
                                chamber['low']['horiz']['90'], chamber['low']['horiz']['180'], chamber['low']['horiz']['270'], chamber['low']['vert']['0'], 
                                chamber['low']['vert']['15'], chamber['low']['vert']['30'], chamber['low']['vert']['45'], chamber['low']['vert']['60'], 
                                chamber['low']['vert']['180'], chamber['low']['vert']['90'], chamber['low']['vert']['-15'], chamber['low']['vert']['-30'], 
                                chamber['low']['vert']['-45'], chamber['low']['vert']['-60'], chamber['low']['vert']['-90']])

# Plot Annie's data to check it worked:
maincolor='blue'
fillcolor='green'


fig = plt.figure(figsize=(15,8))
fig.patch.set_facecolor("white")
plt.plot(chamber['low']['freqs'], (averaged_spectra+dbm_to_eirp+83.1)/24,color=maincolor, label="drone test chamber RFI average", linewidth=2.5)
plt.plot(chamber['low']['freqs'], (all_worst+dbm_to_eirp+83.1)/24, color=fillcolor, alpha=.6, label="drone test chamber RFI bounds")
plt.plot(chamber['low']['freqs'], (all_best+dbm_to_eirp+83.1)/24, color=fillcolor, alpha=.6)

plt.fill_between(chamber['low']['freqs'], (all_worst+dbm_to_eirp+83.1)/24, (all_best+dbm_to_eirp+83.1)/24, color=fillcolor, alpha=.2)
plt.xlim(400,600)
plt.ylim(0, .8)
plt.xlabel("Frequency [MHz]")
plt.ylabel("EIRP [dBm]")
plt.title("M600 Pro RFI Tests: Low-band EMI chamber measurements")
plt.legend()
plt.grid()
plt.tight_layout()


plt.show()

In [ ]:
######################################################################
########## Ambient RFI from TONE ##########
######################################################################


ambdata = np.load('/hirax/GBO_Analysis_Outputs/TONE_ambient_RFI.npz')
ambspec = ambdata['spec']
ambfreq = ambdata['freq']
plot(ambfreq,(ambspec/1E7)-.1,color='black',linewidth='2.5',label='ambient GBO RFI 2021 (2020?)')
plt.show()



######################################################################
########## Flight 625 - drone off data ##########
######################################################################


def get_ranges(centerx,centery,throw):
    xmin = centerx-throw
    xmax = centerx+throw
    ymin = centery-throw
    ymax = centery+throw
    return xmin,xmax,ymin,ymax

def get_indices(x,y,centerx,centery,throw):
    xmin,xmax,ymin,ymax = get_ranges(centerx,centery,throw)
    indi = np.where( (x > xmin) & (x < xmax) & (y > ymin) & (y < ymax))[0]
    return indi

fly = '625'

ffile = glob.glob(fitdir+'*'+fly+'*')[0]
pklfile = glob.glob(pckldir+'*'+fly+'*')[0]

pol, pols, cpols, attn, fi = get_flightinfo(fly)

print(fi, pol, pols)

#  Importing pkl file - specially created to be only background (source off)
with open('/hirax/GBO_Analysis_Outputs/Flight_625_Background_20231114.pkl', 'rb') as inp:
    pconcat = pickle.load(inp)

inds_on_cut=np.intersect1d(np.arange(len(pconcat.t_arr_datetime)),pconcat.inds_on).tolist()
indsuse0 = [j for j in inds_on_cut if str(pconcat.drone_xyz_LC_interp[j,0]) != 'nan']

inds_off_cut=np.intersect1d(np.arange(len(pconcat.t_arr_datetime)),pconcat.inds_off).tolist()
indsoff0 = [j for j in inds_off_cut if str(pconcat.drone_xyz_LC_interp[j,0]) != 'nan']

x = pconcat.drone_xyz_LC_interp[indsuse0,0]
y = pconcat.drone_xyz_LC_interp[indsuse0,1]

xoff = pconcat.drone_xyz_LC_interp[indsoff0,0]
yoff = pconcat.drone_xyz_LC_interp[indsoff0,1]

bgRFI = np.zeros([1024,3,9,16]) # freq, min/max/avg, center/40/80, dish
dish_comb_RFI = np.zeros([1024,3,9]) # freq,min/max/avg, center/40,80

# Inds OFF
# Taking a central core of the flight 

#Taking max, min, and avg for central core over all dishes and plotting 

for di,d in enumerate(pols):  
    for freq_i in range (0,1024):
        try:
            xc = pconcat.G_popt[d,freq_i,1] # x centroid
            yc = pconcat.G_popt[d,freq_i,3] # y centroid
            amp_corr =pconcat.V_bg[indsoff0,freq_i,d]/pconcat.G_popt[d,freq_i,0]

            xp,yp = get_newpointing(xoff,yoff,xc,yc,0.0) 
            # centered at 0,0
            gind = get_indices(xp,yp,0.0,0.0,10)
            bgRFI[freq_i,:,0,d] = [np.nanmin(amp_corr[gind]),np.nanmax(amp_corr[gind]),np.nanmean(amp_corr[gind])] 
            # centered at -80,-80
            gind = get_indices(xp,yp,-80,-80,10)
            bgRFI[freq_i,:,1,d] = [np.nanmin(amp_corr[gind]),np.nanmax(amp_corr[gind]),np.nanmean(amp_corr[gind])] 
            # centered at 80,-80
            gind = get_indices(xp,yp,80,-80,10)
            bgRFI[freq_i,:,2,d] = [np.nanmin(amp_corr[gind]),np.nanmax(amp_corr[gind]),np.nanmean(amp_corr[gind])]
            # centered at -80,80
            gind = get_indices(xp,yp,-80,80,10)
            bgRFI[freq_i,:,3,d] = [np.nanmin(amp_corr[gind]),np.nanmax(amp_corr[gind]),np.nanmean(amp_corr[gind])]
            # centered at 80,80
            gind = get_indices(xp,yp,80,80,10)
            bgRFI[freq_i,:,4,d] = [np.nanmin(amp_corr[gind]),np.nanmax(amp_corr[gind]),np.nanmean(amp_corr[gind])]
            
            # centered at -40,-40
            gind = get_indices(xp,yp,-40,-40,10)
            bgRFI[freq_i,:,5,d] = [np.nanmin(amp_corr[gind]),np.nanmax(amp_corr[gind]),np.nanmean(amp_corr[gind])] 
            # centered at 40,-40
            gind = get_indices(xp,yp,40,-40,10)
            bgRFI[freq_i,:,6,d] = [np.nanmin(amp_corr[gind]),np.nanmax(amp_corr[gind]),np.nanmean(amp_corr[gind])]
            # centered at -40,40
            gind = get_indices(xp,yp,-40,40,10)
            bgRFI[freq_i,:,7,d] = [np.nanmin(amp_corr[gind]),np.nanmax(amp_corr[gind]),np.nanmean(amp_corr[gind])]
            # centered at 40,40
            gind = get_indices(xp,yp,40,40,10)
            bgRFI[freq_i,:,8,d] = [np.nanmin(amp_corr[gind]),np.nanmax(amp_corr[gind]),np.nanmean(amp_corr[gind])]
        except:''

# Averaging all the dishes together

dish_comb_RFI = np.nanmean(bgRFI,axis=-1)

cc = ['black','red','red','red','red','blue','blue','blue','blue']
ll = ['0,0','-80,-80','80,-80','-80,80','80,80','-40,-40','-40,40','40,-40','40,40']
for delt in np.arange(0,9):
    plt.plot(freqs,dish_comb_RFI[:,2,delt],color=cc[delt],label=ll[delt])
plt.ylim(0,0.15)
plt.legend(ncol=3)
plt.ylabel('RFI mean')
plt.xlabel('Frequency [MHz]')
plt.xlim(400,600)
plt.show()
# 0th indenx (directly overhead) clearly has the most RFI; no difference between 40/80





######################################################################
######################################################################
########## Flight 493 - No drone (RTK could be on) ##########
######################################################################
######################################################################

# Importing code from background plot notebook to make plot for drone off data

cdat = '20210824T151215Z'

flys = ['493']

for fly in flys:
    if int(fly) < 537:
        mdir='/hirax/GBO_Aug_2021/TONE_ACQ/'+cdat+'_yale_drone/corr/' # August 2021
        gaindir='/hirax/GBO_Aug_2021/TONE_ACQ/digital_gains/'+cdat+'_yale_drone_yale_digitalgain/' # August 2021
    else:
        mdir='/hirax/GBO_Oct_2021/TONE_ACQ/'+cdat+'_yale_drone/corr/' # Oct 2021
        gaindir='/hirax/GBO_Oct_2021/TONE_ACQ/digital_gains/'+cdat+'_yale_drone_yale_digitalgain/' # Oct 2021

    sdir = os.listdir(mdir)[0]
    datadir=mdir+sdir+'/'
    print(sdir, datadir,gaindir)
    
    dronedir='/hirax/all_drone_data/datcon_csv/'
    dronetest0825=drone.Drone_Data(Drone_Directory=dronedir,FLYTAG='FLY'+fly+'.csv',site_class=gbosite)

    print('DONE reading in drone data')
    print(dronetest0825.t_arr_datetime[0], dronetest0825.t_arr_datetime[-1])

    #pu.Plot_Drone_Coordinates(dronetest0825,coo='xy')#,t_bounds=[tlb,tub])
    #plt.show()
    #pu.Plot_Drone_Coordinates(dronetest0825,coo='xy',t_bounds=[2400,10200])
    print(cdat)
    
    corrtest0825=corr.Corr_Data(Data_Directory=datadir,
                    Gain_Directory=gaindir,site_class=gbosite,
                    crossmap=[])
    


concattest0825=concat.CONCAT(CORRDATCLASS=corrtest0825,DRONEDATCLASS=dronetest0825,\
                             load_yaml=False,traceback=False,save_traceback=False)

#pu.Plot_Time_Series(concattest0825,freqlist=[200,400,832,900]) # previously set to just 700


print(concattest0825.V.shape) # time, frequency, dish

## Calculation below :: 
# Taking max, min, avg for drone off data as a function of frequency over several time indices
# And averaging together 

mins2=[]
maxes2=[]
avg2=[]

tstart = 3990
tend = 4010


mins_fl493 = np.mean(np.min(concattest0825.V[tstart:tend,:,:],axis=0),axis=-1) # find min in time samples
maxes_fl493 = np.mean(np.max(concattest0825.V[tstart:tend,:,:],axis=0),axis=-1) # find max in time samples
avges_fl493 = np.mean(np.mean(concattest0825.V[tstart:tend,:,:],axis=0),axis=-1) # find mean in time samples

plt.plot(freqs,mins_fl493,color=maincolor,linewidth=3,label='drone RFI-telescope average')
plt.fill_between(freqs, mins_fl493, maxes_fl493, color=fillcolor, alpha=0.3)
plt.show()  




In [ ]:
F = 10
colorz=pu.cm_to_discrete(ccm,F)

# set a location to match for scaling: This number at 500 MHz [CHIME freq index of 768]
ff = 294
chfr = 768
refpt = (averaged_spectra[ff]+dbm_to_eirp[ff]+83.1)/24

plt.figure(figsize=(12, 8))

# Annie's plot
maincolor=colorz[9] # 'royalblue'
fillcolor=colorz[9] # '#648FFF'
fig.patch.set_facecolor("white")
plt.plot(chamber['low']['freqs'], (averaged_spectra+dbm_to_eirp+83.1)/24,color=maincolor,linestyle=':', label="Test chamber RFI from drone", linewidth=2.5)
#plt.plot(chamber['low']['freqs'], (all_worst+dbm_to_eirp+83.1)/24, color=fillcolor, alpha=.6)
#plt.plot(chamber['low']['freqs'], (all_best+dbm_to_eirp+83.1)/24, color=fillcolor, alpha=.6)

plt.fill_between(chamber['low']['freqs'], (all_worst+dbm_to_eirp+83.1)/24, (all_best+dbm_to_eirp+83.1)/24, color=fillcolor, alpha=.2)
plt.xlim(400,600)
plt.ylim(0, .8)




#maincolor=colorz[5]
#fillcolor=colorz[5]
#fact=refpt/avges_fl493[chfr]#5E7
#plt.plot(freqs,fact*avges_fl493,color=maincolor,linewidth=3,linestyle=':',label='RTK only average')
#plt.fill_between(freqs, fact*mins_fl493, fact*maxes_fl493, color=fillcolor, alpha=0.3)

#plt.plot(freqs[good_freqs],0.15*np.ones(len(good_freqs)),'k.')

# non-overhead
pp = 5
maincolor=colorz[7] # 'black' '#FE8100'
fillcolor=colorz[7] # '#FFC456'
#fact = 5.0
fact = refpt/dish_comb_RFI[chfr,2,pp]
plt.plot(freqs,fact*dish_comb_RFI[:,0,pp],color=fillcolor, alpha=0.5,linewidth='1')
plt.plot(freqs,fact*dish_comb_RFI[:,2,pp],color=maincolor,linewidth=3,linestyle=':',label='RFI Background during flight (40m away)')
plt.plot(freqs,fact*dish_comb_RFI[:,1,pp],color=fillcolor, alpha=0.5,linewidth='1')
plt.fill_between(freqs, fact*dish_comb_RFI[:,0,pp], fact*dish_comb_RFI[:,1,0], color=fillcolor, alpha=0.3)


maincolor=colorz[0] # 'black' '#FE8100'
fillcolor=colorz[0] # '#FFC456'
#fact = 5.0
fact = refpt/dish_comb_RFI[chfr,2,0]
plt.plot(freqs,fact*dish_comb_RFI[:,0,0],color=fillcolor, alpha=0.5,linewidth='1')
plt.plot(freqs,fact*dish_comb_RFI[:,2,0],color=maincolor,linewidth=3,linestyle=':',label='RFI Background during flight (center)')
plt.plot(freqs,fact*dish_comb_RFI[:,1,0],color=fillcolor, alpha=0.5,linewidth='1')
plt.fill_between(freqs, fact*dish_comb_RFI[:,0,0], fact*dish_comb_RFI[:,1,0], color=fillcolor, alpha=0.3)



maincolor=colorz[3] # '#E50399'
fact = refpt/ambspec[chfr]
plt.plot(ambfreq,fact*ambspec,color=maincolor,linewidth='2.5',label='Ambient GBO RFI spectrum 2020')

plt.legend(fontsize=18)
plt.grid()
plt.tight_layout()
plt.xlabel("Frequency [MHz]",fontsize=18)
plt.ylabel("Arbitrary [Norm. to Test chamber at 500 MHz]",fontsize=18)

plt.show()

In [ ]:
##### KEEP BELOW, MAY HAVE BEEN USED TO GENERATE A SUMMARY FI######

In [ ]:
''''# Generate the average and stddev of the gauss fits per freq, per file -- 

# looks like she wants the format
# [pol, freq, fitpar, 0=value,1=stddev]

Npolflights_fa = ['618','619','625']
Epolflights_fa = ['620']

holderarr = np.zeros([16,1024,6,3])
averaged = np.zeros([16,1024,6,2])

for f,fly in enumerate(Npolflights_fa):
    ffile = glob.glob(fitdir+'*'+fly+'*')[0]
    fits = np.load(ffile)
    holderarr[N_pols,:,:,f] = fits['G_popt'][N_pols,:,0:6]
averaged[N_pols,:,:,0] = np.nanmean(holderarr[N_pols,:,:,:],axis=-1)
averaged[N_pols,:,:,1] = np.nanstd(holderarr[N_pols,:,:,:],axis=-1)

holderarr = np.zeros([16,1024,6,1])

for f,fly in enumerate(Epolflights_fa):
    ffile = glob.glob(fitdir+'*'+fly+'*')[0]
    fits = np.load(ffile)
    print(fits['G_popt'].shape)
    holderarr[E_pols,:,:,f] = fits['G_popt'][E_pols,:,0:6]

averaged[E_pols,:,:,0] = np.nanmean(holderarr[E_pols,:,:,:],axis=-1)
averaged[E_pols,:,:,1] = np.nanstd(holderarr[E_pols,:,:,:],axis=-1)
tmpfitpath='/hirax/GBO_Analysis_Outputs/Averaged_2dGauss_and_Airy_Param.npz'
np.savez(tmpfitpath,averaged=averaged)

### ONLY NEED TO RUN THIS ONCE #########
################ ONLY RUN THIS IF THE FILE DOESN"T EXIST ALREADY ###################
########## Generating the Guassian Fit Parameters and Errors for the Coadded Beams and Saving Them to Files:####

# Generate the average and stddev of the gauss fits per freq, per file -- 

# format
# [pol, freq, fitpar, 0=value,1=stddev]

Npolflights_fa = ['618','619','625']
Epolflights_fa = ['620']

holderarr = np.zeros([16,1024,6,3])
averaged = np.zeros([16,1024,6,2])

for f,fly in enumerate(Npolflights_fa):
    ffile = glob.glob(fitdir+'*'+fly+'*')[0]
    fits = np.load(ffile)
    holderarr[N_pols,:,:,f] = fits['G_popt'][N_pols,:,0:6]
averaged[N_pols,:,:,0] = np.nanmean(holderarr[N_pols,:,:,:],axis=-1)
averaged[N_pols,:,:,1] = np.nanstd(holderarr[N_pols,:,:,:],axis=-1)

holderarr = np.zeros([16,1024,6,1])

for f,fly in enumerate(Epolflights_fa):
    ffile = glob.glob(fitdir+'*'+fly+'*')[0]
    fits = np.load(ffile)
    holderarr[E_pols,:,:,f] = fits['G_popt'][E_pols,:,0:6]

averaged[E_pols,:,:,0] = np.nanmean(holderarr[E_pols,:,:,:],axis=-1)
averaged[E_pols,:,:,1] = np.nanstd(holderarr[E_pols,:,:,:],axis=-1)
tmpfitpath='/hirax/GBO_Analysis_Outputs/Averaged_2dGauss_and_Airy_Param.npz'
np.savez(tmpfitpath,averaged=averaged)


## This performs a least-sq Gaussian fit to the coadded maps and writes to a file

G_popt = np.zeros([16,1024,6,2])
pG=np.array([1.0,0.0,8.0,0.0,8.0,1E-8])

for f,find in enumerate(good_freqs):

    copoldir = 'N'
    pols = N_pols
    picklefile = beamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'.pkl'
    with open(picklefile, 'rb') as inp:
        ptest = pickle.load(inp)

    for j, chind in enumerate(pols): 
        mbx = ptest.x_centers_grid[:,:,chind].flatten()
        mby = ptest.y_centers_grid[:,:,chind].flatten()
        mbV = ptest.V_LC_operation[:,:,0,chind].flatten()
        mbVerr = ptest.V_LC_operation_err[:,:,0,chind].flatten()

        gi = np.where((np.isfinite(mbV) & (np.abs(mbx) < 25) & (np.abs(mby) < 25) & (np.isfinite(mbV)) & (np.isfinite(mbVerr))))[0]
        input_data=np.array([mbx[gi],mby[gi],mbV[gi]])
        gg = least_squares(fu.Gauss_2d_LC_opt,x0=pG,method='trf',args=input_data)
        G_popt[chind,find,:,0] = gg.x
        J = gg.jac
        cov = np.linalg.inv(J.T.dot(J))
        var = np.diagonal(cov)
        G_popt[chind,find,:,1] = np.sqrt(var)     
        
    copoldir = 'E'
    pols = E_pols
    picklefile = beamdir+'Beamcoadd_pol_'+copoldir+'_freq_'+str(find)+'.pkl'
    with open(picklefile, 'rb') as inp:
        ptest = pickle.load(inp)

    for j, chind in enumerate(pols): 
        mbx = ptest.x_centers_grid[:,:,chind].flatten()
        mby = ptest.y_centers_grid[:,:,chind].flatten()
        mbV = ptest.V_LC_operation[:,:,0,chind].flatten()

        gi = np.where((np.isfinite(mbV) & (np.abs(mbx) < 25) & (np.abs(mby) < 25) & (np.isfinite(mbV)) & (np.isfinite(mbVerr))))[0]
        input_data=np.array([mbx[gi],mby[gi],mbV[gi]])
        gg = least_squares(fu.Gauss_2d_LC_opt,x0=pG,method='trf',args=input_data)
        G_popt[chind,find,:,0] = gg.x
        J = gg.jac
        cov = np.linalg.inv(J.T.dot(J))
        var = np.diagonal(cov)
        G_popt[chind,find,:,1] = np.sqrt(var)

tmpfitpath='/hirax/GBO_Analysis_Outputs/Coadded_2dGauss_With_Errors.npz'
np.savez(tmpfitpath,G_popt=G_popt)


''''